In [1]:
#RUN IN SAOD or SAGEO (sageo has a problem with pandas min function)
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
import geopandas as gpd
import scipy.io
from functools import reduce
from pyproj import Proj, transform #can run this fram sageo
import rasterio
import rasterstats
from netCDF4 import Dataset
import shapely
import rioxarray
import xarray
from functools import reduce
import glob
import re
import math
import folium
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.ticker as mticker
from matplotlib.transforms import Affine2D
from matplotlib.lines import Line2D
from geocube.api.core import make_geocube
from shapely.wkt import loads, dumps
from shapely.ops import polygonize, split
from fiona.crs import from_epsg

# from osgeo import ogr
from shapely.geometry import MultiLineString, Point, LineString, Polygon
from shapely import wkt
import sys

pd.set_option('display.max_columns', 200)
shapely.speedups.enable()

# from jupyterthemes import jtplot
# jtplot.style(theme='monokai', context='notebook', ticks=True, grid=False)

%matplotlib inline
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# date_string='24July'
date_string='Spring2021_v5'

In [2]:
#read in fluxgate lintes
flux_lines = gpd.read_file('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/vector_files/Arctic_marine_terminating_fluxgates.gpkg')  

In [3]:
#Make points every distance along your fluxgates or lines
distance=25 #change this distance based on how frequently you want points

easting = []
northing = []
creation = []
region = []

i=1
for i in range(0,len(flux_lines)): #iterate through all your fluxgate lines and create points every distance along them
    for x in range(0,int(flux_lines.loc[i].geometry.length),distance):
        x,y = flux_lines.loc[i].geometry.interpolate(x).xy
        easting.append(x[0])
        northing.append(y[0])
        creation.append(flux_lines.loc[i].creation) #save other attributes
        region.append(flux_lines.loc[i].Region) #save other attributes
#read the data to a pandas dataframe
lists = {'region':region,'creation':creation,'easting':easting,'northing':northing}
df = pd.DataFrame(lists)
#read the data into a GeoDataFrame
flux_gate_points = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.easting, df.northing))
flux_gate_points.crs = {'init' :'epsg:3995'} #define projection
flux_gate_points.to_file('flux_gate_points_25m_'+date_string)#change file name based on distance

In [5]:
#import the files you just wrote above or import them if you are rerunning the script
fluxgate_points_on_glaciers = gpd.read_file('/Users/willkochtitzky/bin/Jupyter_notebooks/Arctic/Arctic_discharge/fluxgate_points_on_glaciers_25m_'+date_string+'/fluxgate_points_on_glaciers_25m_'+date_string+'.shp')

In [6]:
#select by region
CAA_flux_gates = fluxgate_points_on_glaciers[(fluxgate_points_on_glaciers['region']=='QEI')|(fluxgate_points_on_glaciers['region']=='BBI')]
Greenland_flux_gates = fluxgate_points_on_glaciers[fluxgate_points_on_glaciers['region']=='Greenland']
Alaska_flux_gates = fluxgate_points_on_glaciers[fluxgate_points_on_glaciers['region']=='Alaska']
Eastern_Arctic_flux_gates = fluxgate_points_on_glaciers[(fluxgate_points_on_glaciers['region']=='Svalbard')|(fluxgate_points_on_glaciers['region']=='Russia')|(fluxgate_points_on_glaciers['region']=='Jan Mayen')]
Iceland_flux_gates = fluxgate_points_on_glaciers[(fluxgate_points_on_glaciers['region']=='Iceland')|(fluxgate_points_on_glaciers['region']=='Jan Mayen')]

# fluxgate_points_on_glaciers.region.unique()

In [7]:
# #loop through all the ITS_LIVE datasets and export tif files
# datasets = sorted(glob.glob('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/ITS_LIVE/*.nc'))

# for i in range(0,len(datasets)):
#     xds = xarray.open_dataset(datasets[i])
#     xds['v'].rio.to_raster(str(datasets[i]+'_v.tif'))
#     xds['vx'].rio.to_raster(str(datasets[i]+'_vx.tif'))
#     xds['vy'].rio.to_raster(str(datasets[i]+'_vy.tif'))
#     xds['vx_err'].rio.to_raster(str(datasets[i]+'_vx_err.tif'))
#     xds['vy_err'].rio.to_raster(str(datasets[i]+'_vy_err.tif'))
    
# #Next you need to run the following commands in terminal to get them in the right projection
# # for file in *.tif; do gdal_edit.py -a_srs EPSG:3413 $file; done
# # for file in *.tif; do gdalwarp -t_srs EPSG:3995 $file polar_$file; done

In [8]:
#now point query all the tiff data by region - Canada
caa_datasets = sorted(glob.glob('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/ITS_LIVE/polar_CAN*.tif'))
for i in range(0,len(caa_datasets)):
    year = re.search('240_(.*).nc',caa_datasets[i]).group(1)
    data_name= re.search('.nc_(.*).tif',caa_datasets[i]).group(1)
    str(year+'_'+data_name)
    CAA_flux_gates[str(year+'_'+data_name)]=rasterstats.point_query(CAA_flux_gates.geometry,caa_datasets[i],interpolate='nearest')
    
#now point query all the tiff data for each year by region - Canada
caa_datasets = sorted(glob.glob('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/ITS_LIVE/annual_velocity_maps/CAN*_polar.tif'))
for i in range(0,len(caa_datasets)):
    data_name= re.search('240_(.*)_polar.tif',caa_datasets[i]).group(1)
    CAA_flux_gates[str(data_name)]=rasterstats.point_query(CAA_flux_gates.geometry,caa_datasets[i],interpolate='nearest')

#Pull in special velocity data for northern Ellesemere
CAA_flux_gates['Northern_velocity_Ell']=rasterstats.point_query(CAA_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/northern_velocities/Ellesmere_northern_velocity_2018to2020_composite.tif',interpolate='nearest')   
CAA_flux_gates['Northern_velocity_Baf']=rasterstats.point_query(CAA_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/northern_velocities/Southern_baffin_velocity_2019to2020_polar.tif',interpolate='nearest')

CAA_flux_gates['Northern_velocity_Ell_x']=rasterstats.point_query(CAA_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/northern_velocities/Ellesmere_northern_velocity_2018to2020_composite_x.tif',interpolate='nearest')
CAA_flux_gates['Northern_velocity_Baf_x']=rasterstats.point_query(CAA_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/northern_velocities/Southern_baffin_velocity_2019to2020_polar_x.tif',interpolate='nearest')
CAA_flux_gates['Northern_velocity_Ell_y']=rasterstats.point_query(CAA_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/northern_velocities/Ellesmere_northern_velocity_2018to2020_composite_y.tif',interpolate='nearest')
CAA_flux_gates['Northern_velocity_Baf_y']=rasterstats.point_query(CAA_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/northern_velocities/Southern_baffin_velocity_2019to2020_polar_y.tif',interpolate='nearest')


CAA_flux_gates.to_file('CAA_flux_gates_'+date_string+'2020.gpkg', driver="GPKG")#change file name based on distance

/Users/willkochtitzky/opt/miniconda3/envs/old_descartes/lib/python3.7/site-packages/rasterstats/io.py:301: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")
/Users/willkochtitzky/opt/miniconda3/envs/old_descartes/lib/python3.7/site-packages/geopandas/geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [9]:
#now point query all the tiff data by region - Greenland
gre_datasets = sorted(glob.glob('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/ITS_LIVE/polar_GRE*.tif'))
for i in range(0,len(gre_datasets)):
    year = re.search('240_(.*).nc',gre_datasets[i]).group(1)
    data_name= re.search('.nc_(.*).tif',gre_datasets[i]).group(1)
    str(year+'_'+data_name)
    Greenland_flux_gates[str(year+'_'+data_name)]=rasterstats.point_query(Greenland_flux_gates.geometry,gre_datasets[i],interpolate='nearest')
    
#now point query all the tiff data for each year by region - Greenland
gre_datasets = sorted(glob.glob('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/ITS_LIVE/annual_velocity_maps/GRE*_polar.tif'))
for i in range(0,len(gre_datasets)):
    data_name= re.search('240_(.*)_polar.tif',gre_datasets[i]).group(1)
    Greenland_flux_gates[str(data_name)]=rasterstats.point_query(Greenland_flux_gates.geometry,gre_datasets[i],interpolate='nearest')    

#Pull in measures data
gre_measures_datasets = sorted(glob.glob('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Greenland_measures/*polar.tif'))
for i in range(0,len(gre_measures_datasets)):
    data_name= re.search('00_(.*)_v02.1_polar.tif',gre_measures_datasets[i]).group(1)
    data_name = str('GM_'+data_name)
    Greenland_flux_gates[str(data_name)]=rasterstats.point_query(Greenland_flux_gates.geometry,gre_measures_datasets[i],interpolate='nearest')    

    
#Pull in special velocity data for northern Ellesemere
Greenland_flux_gates['Northern_velocity']=rasterstats.point_query(Greenland_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/northern_velocities/Greenland_northern_velocity_2016to2020_composite.tif',interpolate='nearest')
Greenland_flux_gates['Northern_velocity_x']=rasterstats.point_query(Greenland_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/northern_velocities/Greenland_northern_velocity_2016to2020_composite_x.tif',interpolate='nearest')
Greenland_flux_gates['Northern_velocity_y']=rasterstats.point_query(Greenland_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/northern_velocities/Greenland_northern_velocity_2016to2020_composite_y.tif',interpolate='nearest')

Greenland_flux_gates.to_file('GRE_flux_gates_'+date_string+'2020.gpkg', driver="GPKG")#change file name based on distance


In [10]:
#now point query all the tiff data by region - Eastern Arctic
sra_datasets = sorted(glob.glob('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/ITS_LIVE/polar_SRA*.tif'))
for i in range(0,len(sra_datasets)):
    year = re.search('240_(.*).nc',sra_datasets[i]).group(1)
    data_name= re.search('.nc_(.*).tif',sra_datasets[i]).group(1)
    str(year+'_'+data_name)
    Eastern_Arctic_flux_gates[str(year+'_'+data_name)]=rasterstats.point_query(Eastern_Arctic_flux_gates.geometry,sra_datasets[i],interpolate='nearest')
    
#now point query all the tiff data for each year by region - Eastern Arctic
sra_datasets = sorted(glob.glob('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/ITS_LIVE/annual_velocity_maps/SRA*_polar.tif'))
for i in range(0,len(sra_datasets)):
    data_name= re.search('240_(.*)_polar.tif',sra_datasets[i]).group(1)
    Eastern_Arctic_flux_gates[str(data_name)]=rasterstats.point_query(Eastern_Arctic_flux_gates.geometry,sra_datasets[i],interpolate='nearest')    
    
#Read in data from Tazio
Eastern_Arctic_flux_gates['Austfonna_ERS_1995']=rasterstats.point_query(Eastern_Arctic_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/velocity_mapping/Tazio/ERS_1995_dv_norm_m_per_day.tif_polar.tif',interpolate='nearest')
Eastern_Arctic_flux_gates['Svalbard_ALOS1_2007_2011']=rasterstats.point_query(Eastern_Arctic_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/velocity_mapping/Tazio/Glaciers_CCI_IV_RGI07_Svalbard_ALOS1_20070904_20110301.tif_polar.tif',interpolate='nearest')
Eastern_Arctic_flux_gates['Svalbard_ERS1_1992']=rasterstats.point_query(Eastern_Arctic_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/velocity_mapping/Tazio/Glaciers_CCI_IV_RGI07_Svalbard_ERS1_19920103_19920115.tif_polar.tif',interpolate='nearest')
Eastern_Arctic_flux_gates['Svalbard_JERS1_1994_1998']=rasterstats.point_query(Eastern_Arctic_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/velocity_mapping/Tazio/Glaciers_CCI_IV_RGI07_Svalbard_JERS1_19940205_19980326.tif_polar.tif',interpolate='nearest')
Eastern_Arctic_flux_gates['Svalbard_TSX_2008_2012']=rasterstats.point_query(Eastern_Arctic_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/velocity_mapping/Tazio/Glaciers_CCI_IV_RGI07_Svalbard_TSX_20080427_20121226.tif_polar.tif',interpolate='nearest')
Eastern_Arctic_flux_gates['FranzJosefLand_ALOS1_2010_2011']=rasterstats.point_query(Eastern_Arctic_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/velocity_mapping/Tazio/Glaciers_CCI_IV_RGI09_FranzJosefLand_ALOS1_20101222_20110417.tif_polar.tif',interpolate='nearest')
Eastern_Arctic_flux_gates['FranzJosefLand_ALOS2_2016']=rasterstats.point_query(Eastern_Arctic_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/velocity_mapping/Tazio/Glaciers_CCI_IV_RGI09_FranzJosefLand_ALOS2_20160208_20160307.tif_polar.tif',interpolate='nearest')
Eastern_Arctic_flux_gates['FranzJosefLand_JERS1_1996_1998']=rasterstats.point_query(Eastern_Arctic_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/velocity_mapping/Tazio/Glaciers_CCI_IV_RGI09_FranzJosefLand_JERS1_19960404_19980529.tif_polar.tif',interpolate='nearest')
Eastern_Arctic_flux_gates['FranzJosefLand_JERS1_1996_1997']=rasterstats.point_query(Eastern_Arctic_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/velocity_mapping/Tazio/Glaciers_CCI_IV_RGI09_FranzJosefLand_JERS1_19960808_19971011.tif_polar.tif',interpolate='nearest')
Eastern_Arctic_flux_gates['NovayaZemlya_ALOS1_2008_2010']=rasterstats.point_query(Eastern_Arctic_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/velocity_mapping/Tazio/Glaciers_CCI_IV_RGI09_NovayaZemlya_ALOS1_20081211_20100429.tif_polar.tif',interpolate='nearest')
Eastern_Arctic_flux_gates['NovayaZemlya_JERS1_1998']=rasterstats.point_query(Eastern_Arctic_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/velocity_mapping/Tazio/Glaciers_CCI_IV_RGI09_NovayaZemlya_JERS1_19980121_19980325.tif_polar.tif',interpolate='nearest')
Eastern_Arctic_flux_gates['SevernayaZemlya_ALOS1_2010']=rasterstats.point_query(Eastern_Arctic_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/velocity_mapping/Tazio/Glaciers_CCI_IV_RGI09_SevernayaZemlya_ALOS1_20100704_20101217.tif_polar.tif',interpolate='nearest')
Eastern_Arctic_flux_gates['SevernayaZemlya_ERS1_1991_1992']=rasterstats.point_query(Eastern_Arctic_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/velocity_mapping/Tazio/Glaciers_CCI_IV_RGI09_SevernayaZemlya_ERS1_19911018_19920302.tif_polar.tif',interpolate='nearest')

Eastern_Arctic_flux_gates['Northern_velocity']=rasterstats.point_query(Eastern_Arctic_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/northern_velocities/Kara_sea_velocity_2018_polar.tif',interpolate='nearest')
Eastern_Arctic_flux_gates['Northern_velocity_x']=rasterstats.point_query(Eastern_Arctic_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/northern_velocities/Kara_sea_velocity_2018_polar_x.tif',interpolate='nearest')
Eastern_Arctic_flux_gates['Northern_velocity_y']=rasterstats.point_query(Eastern_Arctic_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/northern_velocities/Kara_sea_velocity_2018_polar_y.tif',interpolate='nearest')

Eastern_Arctic_flux_gates.to_file('SRA_flux_gates_'+date_string+'2020.gpkg', driver="GPKG")#change file name based on distance


In [11]:
#now point query all the tiff data by region - Alaska
ala_datasets = sorted(glob.glob('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/ITS_LIVE/polar_ALA*.tif'))
for i in range(0,len(ala_datasets)):
    year = re.search('240_(.*).nc',ala_datasets[i]).group(1)
    data_name= re.search('.nc_(.*).tif',ala_datasets[i]).group(1)
    str(year+'_'+data_name)
    Alaska_flux_gates[str(year+'_'+data_name)]=rasterstats.point_query(Alaska_flux_gates.geometry,ala_datasets[i],interpolate='nearest')
    
#now point query all the tiff data for each year by region - Alaska
ala_datasets = sorted(glob.glob('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/ITS_LIVE/annual_velocity_maps/ALA*_polar.tif'))
for i in range(0,len(sra_datasets)):
    data_name= re.search('240_(.*)_polar.tif',ala_datasets[i]).group(1)
    Alaska_flux_gates[str(data_name)]=rasterstats.point_query(Alaska_flux_gates.geometry,ala_datasets[i],interpolate='nearest')    
    
Alaska_flux_gates.to_file('ALA_flux_gates_'+date_string+'2020.gpkg', driver="GPKG")#change file name based on distance

In [12]:
#now point query all the tiff data by region - Iceland
ice_datasets = sorted(glob.glob('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/ITS_LIVE/polar_ICE*.tif'))
for i in range(0,len(ice_datasets)):
    year = re.search('240_(.*).nc',ice_datasets[i]).group(1)
    data_name= re.search('.nc_(.*).tif',ice_datasets[i]).group(1)
    str(year+'_'+data_name)
    Iceland_flux_gates[str(year+'_'+data_name)]=rasterstats.point_query(Iceland_flux_gates.geometry,ice_datasets[i],interpolate='nearest')
    
#now point query all the tiff data for each year by region - Iceland
ice_datasets = sorted(glob.glob('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/ITS_LIVE/annual_velocity_maps/ICE*_polar.tif'))
for i in range(0,len(ice_datasets)):
    data_name= re.search('240_(.*)_polar.tif',ice_datasets[i]).group(1)
    Iceland_flux_gates[str(data_name)]=rasterstats.point_query(Iceland_flux_gates.geometry,ice_datasets[i],interpolate='nearest')    
    
Iceland_flux_gates.to_file('ICE_flux_gates_'+date_string+'2020.gpkg', driver="GPKG")#change file name based on distance

In [13]:
#Read in GlaThiDa
GlaThiDa= gpd.read_file('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Arctic_GlaThiDa_EPSG3995.gpkg')
#Add GlaThiDa data to the dataset
GlaThiDa['geometry']=GlaThiDa.buffer(100)

In [14]:
#Extract thickness data for each region RGI 3 (BBI) and 4 (QEI)
CAA_flux_gates = gpd.read_file('/Users/willkochtitzky/bin/Jupyter_notebooks/Arctic/Arctic_discharge/CAA_flux_gates_'+date_string+'2020.gpkg')
CAA_BBI_flux_gates = CAA_flux_gates[CAA_flux_gates['region']=='BBI']
CAA_QEI_flux_gates = CAA_flux_gates[CAA_flux_gates['region']=='QEI']
CAA_BBI_flux_gates['Farinotti_thickness']=rasterstats.point_query(CAA_BBI_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/RGI60-04/aa_RGI6_thickness_mosaic.vrt',interpolate='nearest')
CAA_QEI_flux_gates['Farinotti_thickness']=rasterstats.point_query(CAA_QEI_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/RGI60-03/aa_RGI6_thickness_mosaic.vrt',interpolate='nearest')
CAA_QEI_flux_gates['tomo_thickness']=rasterstats.point_query(CAA_QEI_flux_gates.geometry,'/Users/willkochtitzky/Projects/CAA_Velocity_Update_from_Wes/3995_polar_stereographic_all_data/tomo_glacier_thickness.tif',interpolate='nearest')

CAA_BBI_flux_gates['model1_thickness']=rasterstats.point_query(CAA_BBI_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/other_models/results_model_1/RGI60-04/aa_RGI6_thickness_mosaic.vrt',interpolate='nearest')
CAA_QEI_flux_gates['model1_thickness']=rasterstats.point_query(CAA_QEI_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/other_models/results_model_1/RGI60-03/aa_RGI6_thickness_mosaic.vrt',interpolate='nearest')
CAA_BBI_flux_gates['model2_thickness']=rasterstats.point_query(CAA_BBI_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/other_models/results_model_2/RGI60-04/aa_RGI6_thickness_mosaic.vrt',interpolate='nearest')
CAA_QEI_flux_gates['model2_thickness']=rasterstats.point_query(CAA_QEI_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/other_models/results_model_2/RGI60-03/aa_RGI6_thickness_mosaic.vrt',interpolate='nearest')
CAA_BBI_flux_gates['model3_thickness']=rasterstats.point_query(CAA_BBI_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/other_models/results_model_3/RGI60-04/aa_RGI6_thickness_mosaic.vrt',interpolate='nearest')
CAA_QEI_flux_gates['model3_thickness']=rasterstats.point_query(CAA_QEI_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/other_models/results_model_3/RGI60-03/aa_RGI6_thickness_mosaic.vrt',interpolate='nearest')

#Pull in GlaThiDa for QEI
CAA_QEI_flux_gates_with_GlaThiDa = gpd.sjoin(CAA_QEI_flux_gates,GlaThiDa,how='left', op='intersects')
CAA_QEI_flux_gates_with_GlaThiDa=CAA_QEI_flux_gates_with_GlaThiDa[['easting','northing','THICKNESS','THICKNESS_UNCERTAINTY']]#loose all the attributes we don't care about
CAA_QEI_flux_gates_with_GlaThiDa_grouped = CAA_QEI_flux_gates_with_GlaThiDa.groupby(['easting','northing']).mean().reset_index() #take the mean of all the GlaThiDa data within the buffer distance
CAA_QEI_flux_gates=CAA_QEI_flux_gates.merge(CAA_QEI_flux_gates_with_GlaThiDa_grouped,left_on=['easting','northing'],right_on=['easting','northing'])#join the data that have the same easting/northing

#Pull in GlaThiDa for BBI
CAA_BBI_flux_gates_with_GlaThiDa = gpd.sjoin(CAA_BBI_flux_gates,GlaThiDa,how='left', op='intersects')
CAA_BBI_flux_gates_with_GlaThiDa=CAA_BBI_flux_gates_with_GlaThiDa[['easting','northing','THICKNESS','THICKNESS_UNCERTAINTY']]#loose all the attributes we don't care about
CAA_BBI_flux_gates_with_GlaThiDa_grouped = CAA_BBI_flux_gates_with_GlaThiDa.groupby(['easting','northing']).mean().reset_index() #take the mean of all the GlaThiDa data within the buffer distance
CAA_BBI_flux_gates=CAA_BBI_flux_gates.merge(CAA_BBI_flux_gates_with_GlaThiDa_grouped,left_on=['easting','northing'],right_on=['easting','northing'])#join the data that have the same easting/northing

CAA_BBI_flux_gates.to_file('CAN_BBI_flux_gates_thick_'+date_string+'2020.gpkg', driver="GPKG")#change file name based on distance
CAA_QEI_flux_gates.to_file('CAN_QEI_flux_gates_thick_'+date_string+'2020.gpkg', driver="GPKG")#change file name based on distance


In [15]:
#RGI 5
Greenland_flux_gates = gpd.read_file('/Users/willkochtitzky/bin/Jupyter_notebooks/Arctic/Arctic_discharge/GRE_flux_gates_'+date_string+'2020.gpkg')
Greenland_flux_gates['Farinotti_thickness']=rasterstats.point_query(Greenland_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/RGI60-05/aa_RGI6_thickness_mosaic.vrt',interpolate='nearest')
Greenland_flux_gates['model1_thickness']=rasterstats.point_query(Greenland_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/other_models/results_model_1/RGI60-05/aa_RGI6_thickness_mosaic.vrt',interpolate='nearest')
Greenland_flux_gates['model2_thickness']=rasterstats.point_query(Greenland_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/other_models/results_model_2/RGI60-05/aa_RGI6_thickness_mosaic.vrt',interpolate='nearest')

#Pull in GlaThiDa for Greenland
Greenland_flux_gates_with_GlaThiDa = gpd.sjoin(Greenland_flux_gates,GlaThiDa,how='left', op='intersects')
Greenland_flux_gates_with_GlaThiDa=Greenland_flux_gates_with_GlaThiDa[['easting','northing','THICKNESS','THICKNESS_UNCERTAINTY']]#loose all the attributes we don't care about
Greenland_flux_gates_with_GlaThiDa_grouped = Greenland_flux_gates_with_GlaThiDa.groupby(['easting','northing']).mean().reset_index() #take the mean of all the GlaThiDa data within the buffer distance
Greenland_flux_gates=Greenland_flux_gates.merge(Greenland_flux_gates_with_GlaThiDa_grouped,left_on=['easting','northing'],right_on=['easting','northing'])#join the data that have the same easting/northing

Greenland_flux_gates.to_file('GRE_flux_gates_thick_'+date_string+'2020.gpkg', driver="GPKG")#change file name based on distance


In [16]:
#RGI 7 (Svalbard) and 9 (Russia)
Eastern_Arctic_flux_gates = gpd.read_file('/Users/willkochtitzky/bin/Jupyter_notebooks/Arctic/Arctic_discharge/SRA_flux_gates_'+date_string+'2020.gpkg')
Svalbard_flux_gates = Eastern_Arctic_flux_gates[Eastern_Arctic_flux_gates['region']=='Svalbard']
Russia_flux_gates = Eastern_Arctic_flux_gates[Eastern_Arctic_flux_gates['region']=='Russia']

Svalbard_flux_gates['Farinotti_thickness']=rasterstats.point_query(Svalbard_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/RGI60-07/aa_RGI6_thickness_mosaic.vrt',interpolate='nearest')
Russia_flux_gates['Farinotti_thickness']=rasterstats.point_query(Russia_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/RGI60-09/aa_RGI6_thickness_mosaic.vrt',interpolate='nearest')

Svalbard_flux_gates['model1_thickness']=rasterstats.point_query(Svalbard_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/other_models/results_model_1/RGI60-07/aa_RGI6_thickness_mosaic.vrt',interpolate='nearest')
Russia_flux_gates['model1_thickness']=rasterstats.point_query(Russia_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/other_models/results_model_1/RGI60-09/aa_RGI6_thickness_mosaic.vrt',interpolate='nearest')
Svalbard_flux_gates['model2_thickness']=rasterstats.point_query(Svalbard_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/other_models/results_model_2/RGI60-07/aa_RGI6_thickness_mosaic.vrt',interpolate='nearest')
Russia_flux_gates['model2_thickness']=rasterstats.point_query(Russia_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/other_models/results_model_2/RGI60-09/aa_RGI6_thickness_mosaic.vrt',interpolate='nearest')
Svalbard_flux_gates['model3_thickness']=rasterstats.point_query(Svalbard_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/other_models/results_model_3/RGI60-07/aa_RGI6_thickness_mosaic.vrt',interpolate='nearest')
Russia_flux_gates['model3_thickness']=rasterstats.point_query(Russia_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/other_models/results_model_3/RGI60-09/aa_RGI6_thickness_mosaic.vrt',interpolate='nearest')

#Pull in GlaThiDa for Svalbard
Svalbard_flux_gates_with_GlaThiDa = gpd.sjoin(Svalbard_flux_gates,GlaThiDa,how='left', op='intersects')
Svalbard_flux_gates_with_GlaThiDa=Svalbard_flux_gates_with_GlaThiDa[['easting','northing','THICKNESS','THICKNESS_UNCERTAINTY']]#loose all the attributes we don't care about
Svalbard_flux_gates_with_GlaThiDa_grouped = Svalbard_flux_gates_with_GlaThiDa.groupby(['easting','northing']).mean().reset_index() #take the mean of all the GlaThiDa data within the buffer distance
Svalbard_flux_gates=Svalbard_flux_gates.merge(Svalbard_flux_gates_with_GlaThiDa_grouped,left_on=['easting','northing'],right_on=['easting','northing'])#join the data that have the same easting/northing
#Pull in GlaThiDa for Russia
Russia_flux_gates_with_GlaThiDa = gpd.sjoin(Russia_flux_gates,GlaThiDa,how='left', op='intersects')
Russia_flux_gates_with_GlaThiDa=Russia_flux_gates_with_GlaThiDa[['easting','northing','THICKNESS','THICKNESS_UNCERTAINTY']]#loose all the attributes we don't care about
Russia_flux_gates_with_GlaThiDa_grouped = Russia_flux_gates_with_GlaThiDa.groupby(['easting','northing']).mean().reset_index() #take the mean of all the GlaThiDa data within the buffer distance
Russia_flux_gates=Russia_flux_gates.merge(Russia_flux_gates_with_GlaThiDa_grouped,left_on=['easting','northing'],right_on=['easting','northing'])#join the data that have the same easting/northing

#Pull in Andrey's thickness obs for Russia
Russia_thickness = gpd.read_file('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/Russia/Russia_thickness_merged_EPSG3995.geojson')
Russia_thickness['geometry']=Russia_thickness.buffer(100)
Russia_flux_gates_with_Russia_thickness = gpd.sjoin(Russia_flux_gates,Russia_thickness,how='left', op='intersects')
Russia_flux_gates_with_Russia_thickness=Russia_flux_gates_with_Russia_thickness[['easting','northing','Russian_thickness']]#loose all the attributes we don't care about
Russia_flux_gates_with_Russia_thickness_grouped = Russia_flux_gates_with_Russia_thickness.groupby(['easting','northing']).mean().reset_index() #take the mean of all the GlaThiDa data within the buffer distance
Russia_flux_gates=Russia_flux_gates.merge(Russia_flux_gates_with_Russia_thickness_grouped,left_on=['easting','northing'],right_on=['easting','northing'])#join the data that have the same easting/northing

print('nans before RES data',Russia_flux_gates['Russian_thickness'].isna().sum())

#Read in RES data from Toby and Julian
Russia_thickness = gpd.read_file('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/RES_data/RES_ice_thickness.gpkg')
Russia_thickness['geometry']=Russia_thickness.buffer(100)
Russia_flux_gates_with_Russia_thickness = gpd.sjoin(Russia_flux_gates,Russia_thickness,how='left', op='intersects')
Russia_flux_gates_with_Russia_thickness=Russia_flux_gates_with_Russia_thickness[['easting','northing','thickness(m)']]#loose all the attributes we don't care about
Russia_flux_gates_with_Russia_thickness_grouped = Russia_flux_gates_with_Russia_thickness.groupby(['easting','northing']).mean().reset_index() #take the mean of all the GlaThiDa data within the buffer distance
Russia_flux_gates=Russia_flux_gates.merge(Russia_flux_gates_with_Russia_thickness_grouped,left_on=['easting','northing'],right_on=['easting','northing'])#join the data that have the same easting/northing

Russia_flux_gates['Russian_thickness'].fillna(Russia_flux_gates['thickness(m)'],inplace=True) #fill in values from Julian and Toby
Russia_flux_gates = Russia_flux_gates.drop(columns='thickness(m)')  #thickness(m) column because we dont need it now
print('nans after RES data',Russia_flux_gates['Russian_thickness'].isna().sum())

Svalbard_flux_gates.to_file('Svalbard_flux_gates_thick_'+date_string+'2020.gpkg', driver="GPKG")#change file name based on distance
Russia_flux_gates.to_file('Russia_flux_gates_thick_'+date_string+'2020.gpkg', driver="GPKG")#change file name based on distance


/Users/willkochtitzky/opt/miniconda3/envs/old_descartes/lib/python3.7/site-packages/geopandas/geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


nans before RES data 88702
nans after RES data 79786


In [17]:
#RGI 1
Alaska_flux_gates = gpd.read_file('/Users/willkochtitzky/bin/Jupyter_notebooks/Arctic/Arctic_discharge/ALA_flux_gates_'+date_string+'2020.gpkg')
Alaska_flux_gates['Farinotti_thickness']=rasterstats.point_query(Alaska_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/RGI60-01/aa_RGI6_thickness_mosaic.vrt',interpolate='nearest')

Alaska_flux_gates['model1_thickness']=rasterstats.point_query(Alaska_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/other_models/results_model_1/RGI60-01/aa_RGI6_thickness_mosaic.vrt',interpolate='nearest')
Alaska_flux_gates['model2_thickness']=rasterstats.point_query(Alaska_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/other_models/results_model_2/RGI60-01/aa_RGI6_thickness_mosaic.vrt',interpolate='nearest')
Alaska_flux_gates['model3_thickness']=rasterstats.point_query(Alaska_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/other_models/results_model_3/RGI60-01/aa_RGI6_thickness_mosaic.vrt',interpolate='nearest')



#Pull in GlaThiDa for Alaska
Alaska_flux_gates_with_GlaThiDa = gpd.sjoin(Alaska_flux_gates,GlaThiDa,how='left', op='intersects')
Alaska_flux_gates_with_GlaThiDa=Alaska_flux_gates_with_GlaThiDa[['easting','northing','THICKNESS','THICKNESS_UNCERTAINTY']]#loose all the attributes we don't care about
Alaska_flux_gates_with_GlaThiDa_grouped = Alaska_flux_gates_with_GlaThiDa.groupby(['easting','northing']).mean().reset_index() #take the mean of all the GlaThiDa data within the buffer distance
Alaska_flux_gates=Alaska_flux_gates.merge(Alaska_flux_gates_with_GlaThiDa_grouped,left_on=['easting','northing'],right_on=['easting','northing'])#join the data that have the same easting/northing

Alaska_flux_gates.to_file('ALA_flux_gates_thick_'+date_string+'2020.gpkg', driver="GPKG")#change file name based on distance

In [18]:
#RGI 6
Iceland_flux_gates = gpd.read_file('/Users/willkochtitzky/bin/Jupyter_notebooks/Arctic/Arctic_discharge/ICE_flux_gates_'+date_string+'2020.gpkg')
Iceland_flux_gates['Farinotti_thickness']=rasterstats.point_query(Iceland_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/RGI60-06/aa_RGI6_thickness_mosaic.vrt',interpolate='nearest')

Iceland_flux_gates['model1_thickness']=rasterstats.point_query(Iceland_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/other_models/results_model_1/RGI60-06/aa_RGI6_thickness_mosaic.vrt',interpolate='nearest')
Iceland_flux_gates['model2_thickness']=rasterstats.point_query(Iceland_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/other_models/results_model_2/RGI60-06/aa_RGI6_thickness_mosaic.vrt',interpolate='nearest')
Iceland_flux_gates['model3_thickness']=rasterstats.point_query(Iceland_flux_gates.geometry,'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/other_models/results_model_3/RGI60-06/aa_RGI6_thickness_mosaic.vrt',interpolate='nearest')



#Pull in GlaThiDa for Iceland
Iceland_flux_gates_with_GlaThiDa = gpd.sjoin(Iceland_flux_gates,GlaThiDa,how='left', op='intersects')
Iceland_flux_gates_with_GlaThiDa=Iceland_flux_gates_with_GlaThiDa[['easting','northing','THICKNESS','THICKNESS_UNCERTAINTY']]#loose all the attributes we don't care about
Iceland_flux_gates_with_GlaThiDa_grouped = Iceland_flux_gates_with_GlaThiDa.groupby(['easting','northing']).mean().reset_index() #take the mean of all the GlaThiDa data within the buffer distance
Iceland_flux_gates=Iceland_flux_gates.merge(Iceland_flux_gates_with_GlaThiDa_grouped,left_on=['easting','northing'],right_on=['easting','northing'])#join the data that have the same easting/northing

Iceland_flux_gates.to_file('ICE_flux_gates_thick_'+date_string+'2020.gpkg', driver="GPKG")#change file name based on distance



In [2]:
#Read in all the files with thickness data

CAA_BBI_flux_gates = gpd.read_file('/Users/willkochtitzky/bin/Jupyter_notebooks/Arctic/Arctic_discharge/CAN_BBI_flux_gates_thick_'+date_string+'2020.gpkg')
CAA_QEI_flux_gates = gpd.read_file('/Users/willkochtitzky/bin/Jupyter_notebooks/Arctic/Arctic_discharge/CAN_QEI_flux_gates_thick_'+date_string+'2020.gpkg')
Greenland_flux_gates = gpd.read_file('/Users/willkochtitzky/bin/Jupyter_notebooks/Arctic/Arctic_discharge/GRE_flux_gates_thick_'+date_string+'2020.gpkg')
Svalbard_flux_gates = gpd.read_file('/Users/willkochtitzky/bin/Jupyter_notebooks/Arctic/Arctic_discharge/Svalbard_flux_gates_thick_'+date_string+'2020.gpkg')
Russia_flux_gates = gpd.read_file('/Users/willkochtitzky/bin/Jupyter_notebooks/Arctic/Arctic_discharge/Russia_flux_gates_thick_'+date_string+'2020.gpkg')
Alaska_flux_gates = gpd.read_file('/Users/willkochtitzky/bin/Jupyter_notebooks/Arctic/Arctic_discharge/ALA_flux_gates_thick_'+date_string+'2020.gpkg')
Iceland_flux_gates = gpd.read_file('/Users/willkochtitzky/bin/Jupyter_notebooks/Arctic/Arctic_discharge/ICE_flux_gates_thick_'+date_string+'2020.gpkg')

CAA_BBI_flux_gates['region']='Canada'
CAA_QEI_flux_gates['region']='Canada'
Greenland_flux_gates['region']='Greenland'
Svalbard_flux_gates['region']='Svalbard'
Russia_flux_gates['region']='Russia'
Alaska_flux_gates['region']='Alaska'
Iceland_flux_gates['region']='Iceland'

Arctic_discharge = pd.concat([CAA_BBI_flux_gates,CAA_QEI_flux_gates,Greenland_flux_gates,Svalbard_flux_gates,Russia_flux_gates,Alaska_flux_gates,Iceland_flux_gates])
Arctic_discharge['length']=25 #length of point spacing from fluxlines
Arctic_discharge['Farinotti_thickness'] = Arctic_discharge['Farinotti_thickness'].fillna(0)

#Fix RGI numbers
Arctic_discharge['RGIId_original']=Arctic_discharge['RGIId']
Arctic_discharge['RGIId']=Arctic_discharge['RGIid_2']
Arctic_discharge=Arctic_discharge.drop('RGIid_2',axis=1)

#fill glaciers that have no thickness data due to mistakes in Farinotti and RGI
mask = Arctic_discharge['RGIId']=='RGI60-05.07646'
Arctic_discharge.loc[mask, 'Farinotti_thickness']=Arctic_discharge.loc[mask, 'Farinotti_thickness'].replace(0,70)
mask = Arctic_discharge['RGIId']=='RGI60-05.10938'
Arctic_discharge.loc[mask, 'Farinotti_thickness']=Arctic_discharge.loc[mask, 'Farinotti_thickness'].replace(0,170)
mask = Arctic_discharge['RGIId']=='RGI60-09.01069'
Arctic_discharge.loc[mask, 'Farinotti_thickness']=Arctic_discharge.loc[mask, 'Farinotti_thickness'].replace(0,120)
mask = Arctic_discharge['RGIId']=='RGI60-05.10945'
Arctic_discharge.loc[mask, 'Farinotti_thickness']=Arctic_discharge.loc[mask, 'Farinotti_thickness'].replace(0,170)
mask = Arctic_discharge['RGIId']=='RGI60-10.00004'
Arctic_discharge.loc[mask, 'Farinotti_thickness']=Arctic_discharge.loc[mask, 'Farinotti_thickness'].replace(0,50)

#convert the dataframe to all numbers (some columns are objects not float 64)
for i in range(0, 85):
    Arctic_discharge.iloc[:,i] = pd.to_numeric(Arctic_discharge.iloc[:,i], errors='ignore')
    
Arctic_discharge['Northern_velocity'].fillna(Arctic_discharge['Northern_velocity_Baf'], inplace=True)
Arctic_discharge['Northern_velocity'].fillna(Arctic_discharge['Northern_velocity_Ell'], inplace=True)

Arctic_discharge['Northern_velocity_x'].fillna(Arctic_discharge['Northern_velocity_Baf_x'], inplace=True)
Arctic_discharge['Northern_velocity_x'].fillna(Arctic_discharge['Northern_velocity_Ell_x'], inplace=True)

Arctic_discharge['Northern_velocity_y'].fillna(Arctic_discharge['Northern_velocity_Baf_y'], inplace=True)
Arctic_discharge['Northern_velocity_y'].fillna(Arctic_discharge['Northern_velocity_Ell_y'], inplace=True)

/Users/willkochtitzky/opt/miniconda3/envs/old_descartes/lib/python3.7/site-packages/ipykernel_launcher.py:19: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [3]:
#clean up mistakes in the ITS_LIVE error record (where there is an error value but no observation, this messes up the northern extras)
no_vel_obs_idx = Arctic_discharge['0000_v'].isna()
Arctic_discharge['0000_vx_err'][no_vel_obs_idx]=np.nan
Arctic_discharge['0000_vy_err'][no_vel_obs_idx]=np.nan

#calculate ITS_LIVE error
Arctic_discharge['0000_v_err']=np.sqrt(Arctic_discharge['0000_vy_err']**2+Arctic_discharge['0000_vx_err']**2)
Arctic_discharge = Arctic_discharge.reset_index()

/Users/willkochtitzky/opt/miniconda3/envs/old_descartes/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/willkochtitzky/opt/miniconda3/envs/old_descartes/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [4]:
#Pull in data from Tazio
#Fill errs with nan
Arctic_discharge['Austfonna_ERS_1995_err'] = np.nan
Arctic_discharge['Svalbard_ALOS1_2007_2011_err'] = np.nan
Arctic_discharge['Svalbard_ERS1_1992_err'] = np.nan
Arctic_discharge['Svalbard_JERS1_1994_1998_err'] = np.nan
Arctic_discharge['Svalbard_TSX_2008_2012_err'] = np.nan
Arctic_discharge['FranzJosefLand_ALOS1_2010_2011_err'] = np.nan
Arctic_discharge['FranzJosefLand_ALOS2_2016_err'] = np.nan
Arctic_discharge['FranzJosefLand_JERS1_1996_1998_err'] = np.nan
Arctic_discharge['FranzJosefLand_JERS1_1996_1997_err'] = np.nan
Arctic_discharge['NovayaZemlya_ALOS1_2008_2010_err'] = np.nan
Arctic_discharge['NovayaZemlya_JERS1_1998_err'] = np.nan
Arctic_discharge['SevernayaZemlya_ALOS1_2010_err'] = np.nan
Arctic_discharge['SevernayaZemlya_ERS1_1991_1992_err'] = np.nan
#convert to float so that the below code works
Arctic_discharge['Austfonna_ERS_1995'] = Arctic_discharge['Austfonna_ERS_1995'].astype(float)
Arctic_discharge['Svalbard_ALOS1_2007_2011'] = Arctic_discharge['Svalbard_ALOS1_2007_2011'].astype(float)
Arctic_discharge['Svalbard_ERS1_1992'] = Arctic_discharge['Svalbard_ERS1_1992'].astype(float)
Arctic_discharge['Svalbard_JERS1_1994_1998'] = Arctic_discharge['Svalbard_JERS1_1994_1998'].astype(float)
Arctic_discharge['Svalbard_TSX_2008_2012'] = Arctic_discharge['Svalbard_TSX_2008_2012'].astype(float)
Arctic_discharge['FranzJosefLand_ALOS1_2010_2011'] = Arctic_discharge['FranzJosefLand_ALOS1_2010_2011'].astype(float)
Arctic_discharge['FranzJosefLand_ALOS2_2016'] = Arctic_discharge['FranzJosefLand_ALOS2_2016'].astype(float)
Arctic_discharge['FranzJosefLand_JERS1_1996_1998'] = Arctic_discharge['FranzJosefLand_JERS1_1996_1998'].astype(float)
Arctic_discharge['FranzJosefLand_JERS1_1996_1997'] = Arctic_discharge['FranzJosefLand_JERS1_1996_1997'].astype(float)
Arctic_discharge['NovayaZemlya_ALOS1_2008_2010'] = Arctic_discharge['NovayaZemlya_ALOS1_2008_2010'].astype(float)
Arctic_discharge['NovayaZemlya_JERS1_1998'] = Arctic_discharge['NovayaZemlya_JERS1_1998'].astype(float)
Arctic_discharge['SevernayaZemlya_ALOS1_2010'] = Arctic_discharge['SevernayaZemlya_ALOS1_2010'].astype(float)
Arctic_discharge['SevernayaZemlya_ERS1_1991_1992'] = Arctic_discharge['SevernayaZemlya_ERS1_1991_1992'].astype(float)

#incorporate data from Tazio
for i in range(0,len(Arctic_discharge)):    
    if Arctic_discharge['Austfonna_ERS_1995'][i]>0:
        Arctic_discharge['Austfonna_ERS_1995_err'][i] = 10
    if Arctic_discharge['Svalbard_ALOS1_2007_2011'][i]>0:
        Arctic_discharge['Svalbard_ALOS1_2007_2011_err'][i] = 10
    if Arctic_discharge['Svalbard_ERS1_1992'][i]>0:
        Arctic_discharge['Svalbard_ERS1_1992_err'][i] = 40
    if Arctic_discharge['Svalbard_JERS1_1994_1998'][i]>0:
        Arctic_discharge['Svalbard_JERS1_1994_1998_err'][i] = 20
    if Arctic_discharge['Svalbard_TSX_2008_2012'][i]>0:
        Arctic_discharge['Svalbard_TSX_2008_2012_err'][i] = 10
    if Arctic_discharge['FranzJosefLand_ALOS1_2010_2011'][i]>0:
        Arctic_discharge['FranzJosefLand_ALOS1_2010_2011_err'][i] = 10
    if Arctic_discharge['FranzJosefLand_ALOS2_2016'][i]>0:
        Arctic_discharge['FranzJosefLand_ALOS2_2016_err'][i] = 10
    if Arctic_discharge['FranzJosefLand_JERS1_1996_1998'][i]>0:
        Arctic_discharge['FranzJosefLand_JERS1_1996_1998_err'][i] = 20
    if Arctic_discharge['FranzJosefLand_JERS1_1996_1997'][i]>0:
        Arctic_discharge['FranzJosefLand_JERS1_1996_1997_err'][i] = 20
    if Arctic_discharge['NovayaZemlya_ALOS1_2008_2010'][i]>0:
        Arctic_discharge['NovayaZemlya_ALOS1_2008_2010_err'][i] = 10
    if Arctic_discharge['NovayaZemlya_JERS1_1998'][i]>0:
        Arctic_discharge['NovayaZemlya_JERS1_1998_err'][i] = 20
    if Arctic_discharge['SevernayaZemlya_ALOS1_2010'][i]>0:
        Arctic_discharge['SevernayaZemlya_ALOS1_2010_err'][i] = 10
    if Arctic_discharge['SevernayaZemlya_ERS1_1991_1992'][i]>0:
        Arctic_discharge['SevernayaZemlya_ERS1_1991_1992_err'][i] = 40

/Users/willkochtitzky/opt/miniconda3/envs/old_descartes/lib/python3.7/site-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/willkochtitzky/opt/miniconda3/envs/old_descartes/lib/python3.7/site-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/willkochtitzky/opt/miniconda3/envs/old_descartes/lib/python3.7/site-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a

In [5]:
GM_errx = ['GM_2000_2001_ex', 'GM_2005_2006_ex', 'GM_2006_2007_ex', 'GM_2007_2008_ex', 'GM_2008_2009_ex', 'GM_2009_2010_ex', 'GM_2012_2013_ex', 'GM_2014_2015_ex', 'GM_2015_2016_ex', 'GM_2016_2017_ex', 'GM_2017_2018_ex']
GM_erry = ['GM_2000_2001_ey', 'GM_2005_2006_ey', 'GM_2006_2007_ey', 'GM_2007_2008_ey', 'GM_2008_2009_ey', 'GM_2009_2010_ey', 'GM_2012_2013_ey', 'GM_2014_2015_ey', 'GM_2015_2016_ey', 'GM_2016_2017_ey', 'GM_2017_2018_ey']
GM_v = ['GM_2000_2001_vv', 'GM_2005_2006_vv', 'GM_2006_2007_vv', 'GM_2007_2008_vv', 'GM_2008_2009_vv', 'GM_2009_2010_vv', 'GM_2012_2013_vv', 'GM_2014_2015_vv', 'GM_2015_2016_vv', 'GM_2016_2017_vv', 'GM_2017_2018_vv']
GM_vx = ['GM_2000_2001_vx', 'GM_2005_2006_vx', 'GM_2006_2007_vx', 'GM_2007_2008_vx', 'GM_2008_2009_vx', 'GM_2009_2010_vx', 'GM_2012_2013_vx', 'GM_2014_2015_vx', 'GM_2015_2016_vx', 'GM_2016_2017_vx', 'GM_2017_2018_vx']
GM_vy = ['GM_2000_2001_vy', 'GM_2005_2006_vy', 'GM_2006_2007_vy', 'GM_2007_2008_vy', 'GM_2008_2009_vy', 'GM_2009_2010_vy', 'GM_2012_2013_vy', 'GM_2014_2015_vy', 'GM_2015_2016_vy', 'GM_2016_2017_vy', 'GM_2017_2018_vy']


IL_v_err = ['2001_v_err', '2006_v_err', '2007_v_err', '2008_v_err', '2009_v_err', '2010_v_err', '2013_v_err', '2015_v_err', '2016_v_err', '2017_v_err', '2018_v_err']
IL_v = ['2001_v', '2006_v', '2007_v', '2008_v', '2009_v', '2010_v', '2013_v', '2015_v', '2016_v', '2017_v', '2018_v']
IL_vx = ['2001_vx', '2006_vx', '2007_vx', '2008_vx', '2009_vx', '2010_vx', '2013_vx', '2015_vx', '2016_vx', '2017_vx', '2018_vx']
IL_vy = ['2001_vy', '2006_vy', '2007_vy', '2008_vy', '2009_vy', '2010_vy', '2013_vy', '2015_vy', '2016_vy', '2017_vy', '2018_vy']


# %matplotlib notebook
for n in range(0,len(GM_errx)):
    GM_verr_field = 'GM_' + str(IL_v[n]) + '_err'
    GM_err_ratio_field = 'GM_' + str(IL_v[n]) + '_err_ratio'
    Arctic_discharge[GM_verr_field] = np.sqrt(Arctic_discharge[GM_errx[n]]**2+Arctic_discharge[GM_erry[n]]**2)
    Arctic_discharge[GM_err_ratio_field] = Arctic_discharge[GM_verr_field]/Arctic_discharge[GM_v[n]]
#     plot_Arctic_discharge = Arctic_discharge[(Arctic_discharge[GM_err_ratio_field]<.5)&((Arctic_discharge[IL_v_err[n]]/Arctic_discharge[IL_v[n]])<.5)]
#     plt.figure()
#     plt.scatter(plot_Arctic_discharge[GM_v[n]],plot_Arctic_discharge[IL_v[n]])
#     plt.xlabel(GM_v[n])
#     plt.ylabel(IL_v[n])
#     plt.show()
#     plt.savefig(IL_v[n])
    
    for i in range(0,len(Arctic_discharge)):
        if (Arctic_discharge[GM_err_ratio_field][i]<Arctic_discharge[IL_v_err[n]][i]):
            Arctic_discharge[IL_v_err[n]][i] = Arctic_discharge[GM_verr_field][i]
            Arctic_discharge[IL_v[n]][i] = Arctic_discharge[GM_v[n]][i]
            Arctic_discharge[IL_vx[n]][i] = Arctic_discharge[GM_vx[n]][i]
            Arctic_discharge[IL_vy[n]][i] = Arctic_discharge[GM_vy[n]][i]

/Users/willkochtitzky/opt/miniconda3/envs/old_descartes/lib/python3.7/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/willkochtitzky/opt/miniconda3/envs/old_descartes/lib/python3.7/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/willkochtitzky/opt/miniconda3/envs/old_descartes/lib/python3.7/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a

In [6]:
v_vals = ['1999_v','2000_v','2001_v','2002_v','2003_v','2004_v','2005_v','2006_v','2007_v','2008_v','2009_v','2010_v','2011_v','2012_v','2013_v','2014_v','2015_v','2016_v','2017_v','2018_v']
err_vals = ['1999_v_err','2000_v_err','2001_v_err','2002_v_err','2003_v_err','2004_v_err','2005_v_err','2006_v_err','2007_v_err','2008_v_err','2009_v_err','2010_v_err','2011_v_err','2012_v_err','2013_v_err','2014_v_err','2015_v_err','2016_v_err','2017_v_err','2018_v_err']

for i in range(0,len(v_vals)):
    ratio_vals=str(err_vals[i])+'_ratio'
    Arctic_discharge[ratio_vals]=Arctic_discharge[err_vals[i]]/Arctic_discharge[v_vals[i]]
    
Arctic_discharge['2000_2010_min_vel_ratio'] = Arctic_discharge[['2000_v_err_ratio', '2001_v_err_ratio', '2002_v_err_ratio', '2003_v_err_ratio', '2004_v_err_ratio', '2005_v_err_ratio', '2006_v_err_ratio', '2007_v_err_ratio', '2008_v_err_ratio', '2009_v_err_ratio']].min(axis=1,skipna=True)
Arctic_discharge['2010_2018_min_vel_ratio'] = Arctic_discharge[['2010_v_err_ratio', '2011_v_err_ratio', '2012_v_err_ratio', '2013_v_err_ratio', '2014_v_err_ratio', '2015_v_err_ratio', '2016_v_err_ratio', '2017_v_err_ratio', '2018_v_err_ratio']].min(axis=1,skipna=True)

# for i in range(0,len(Arctic_discharge)):
#     if Arctic_discharge['2000_2010_min_vel_ratio'][i]<0.5:

In [7]:
#Figure out how many good obs we have, so we can throw out bad obs
Arctic_discharge['num_bad_obs_snr_too_high_early'] = 0
Arctic_discharge['num_good_obs_early'] = 0
Arctic_discharge['num_nan_obs_early'] = 0

for i in range(0,len(Arctic_discharge)):
    for err_ratio_field in ['2000_v_err_ratio', '2001_v_err_ratio', '2002_v_err_ratio', '2003_v_err_ratio', '2004_v_err_ratio', '2005_v_err_ratio', '2006_v_err_ratio', '2007_v_err_ratio', '2008_v_err_ratio', '2009_v_err_ratio']:
        if math.isnan(Arctic_discharge[err_ratio_field][i]):
            Arctic_discharge['num_nan_obs_early'][i] = Arctic_discharge['num_nan_obs_early'][i]+1
        elif Arctic_discharge[err_ratio_field][i] >= 1:
            Arctic_discharge['num_bad_obs_snr_too_high_early'][i] = Arctic_discharge['num_bad_obs_snr_too_high_early'][i]+1
        elif Arctic_discharge[err_ratio_field][i] < 1:
            Arctic_discharge['num_good_obs_early'][i] = Arctic_discharge['num_good_obs_early'][i]+1
            
#put a nan in places where the errors are high and we have good data elsewhere.
for i in range(0,len(Arctic_discharge)):
    if Arctic_discharge['num_good_obs_early'][i]>=1:
        for err_ratio_field in ['2000_v_err_ratio', '2001_v_err_ratio', '2002_v_err_ratio', '2003_v_err_ratio', '2004_v_err_ratio', '2005_v_err_ratio', '2006_v_err_ratio', '2007_v_err_ratio', '2008_v_err_ratio', '2009_v_err_ratio']:
            if Arctic_discharge[err_ratio_field][i]>=1:
                Arctic_discharge[err_ratio_field[0:6]][i]=np.nan #get the velocity field and set to nan
                Arctic_discharge[err_ratio_field[0:6]][i]=np.nan # get the error field and set to nan
                Arctic_discharge[err_ratio_field[0:6]+'y'][i]=np.nan # get the x field and set to nan
                Arctic_discharge[err_ratio_field[0:6]+'y'][i]=np.nan # get the y field and set to nan
                
Arctic_discharge['num_bad_obs_snr_too_high_late'] = 0
Arctic_discharge['num_good_obs_late'] = 0
Arctic_discharge['num_nan_obs_late'] = 0

for i in range(0,len(Arctic_discharge)):
    for err_ratio_field in ['2000_v_err_ratio', '2001_v_err_ratio', '2002_v_err_ratio', '2003_v_err_ratio', '2004_v_err_ratio', '2005_v_err_ratio', '2006_v_err_ratio', '2007_v_err_ratio', '2008_v_err_ratio', '2009_v_err_ratio']:
        if math.isnan(Arctic_discharge[err_ratio_field][i]):
            Arctic_discharge['num_nan_obs_late'][i] = Arctic_discharge['num_nan_obs_late'][i]+1
        elif Arctic_discharge[err_ratio_field][i] >= 1:
            Arctic_discharge['num_bad_obs_snr_too_high_late'][i] = Arctic_discharge['num_bad_obs_snr_too_high_late'][i]+1
        elif Arctic_discharge[err_ratio_field][i] < 1:
            Arctic_discharge['num_good_obs_late'][i] = Arctic_discharge['num_good_obs_late'][i]+1
            
#put a nan in places where the errors are high and we have good data elsewhere.
for i in range(0,len(Arctic_discharge)):
    if Arctic_discharge['num_good_obs_late'][i]>=1:
        for err_ratio_field in ['2010_v_err_ratio', '2011_v_err_ratio', '2012_v_err_ratio', '2013_v_err_ratio', '2014_v_err_ratio', '2015_v_err_ratio', '2016_v_err_ratio', '2017_v_err_ratio', '2018_v_err_ratio']:
            if Arctic_discharge[err_ratio_field][i]>=1:
                Arctic_discharge[err_ratio_field[0:6]][i]=np.nan #get the velocity field and set to nan
                Arctic_discharge[err_ratio_field[0:6]][i]=np.nan # get the error field and set to nan
                Arctic_discharge[err_ratio_field[0:6]+'y'][i]=np.nan # get the x field and set to nan
                Arctic_discharge[err_ratio_field[0:6]+'y'][i]=np.nan # get the y field and set to nan


/Users/willkochtitzky/opt/miniconda3/envs/old_descartes/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/Users/willkochtitzky/opt/miniconda3/envs/old_descartes/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/Users/willkochtitzky/opt/miniconda3/envs/old_descartes/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-do

In [8]:
#Create velocity products
#Fill the uncertainty from custom velocity maps with 0.4 m/yr (bedrock movement in these maps)
northern_vel_idx = Arctic_discharge['0000_v'].isna()&Arctic_discharge['Northern_velocity'].notna()
Arctic_discharge['Northern_vel_err']=np.nan
Arctic_discharge['Northern_vel_err'][northern_vel_idx]=0.4
#insert northern_velocities where there are holes in ITS_LIVE
Arctic_discharge['0000_v'].fillna(Arctic_discharge['Northern_velocity'], inplace=True)
#calculate errors
Arctic_discharge['0000_v_err'].fillna(Arctic_discharge['Northern_vel_err'], inplace=True)

#figure out the direction of flow (use mean)
Arctic_discharge['2000_2010_mean_x'] = Arctic_discharge[['2000_vx','2001_vx','2002_vx','2003_vx','2004_vx','2005_vx','2006_vx','2007_vx','2008_vx','2009_vx']].mean(axis=1,skipna=True)
Arctic_discharge['2000_2010_mean_y'] = Arctic_discharge[['2000_vy','2001_vy','2002_vy','2003_vy','2004_vy','2005_vy','2006_vy','2007_vy','2008_vy','2009_vy']].mean(axis=1,skipna=True)
Arctic_discharge['2010_2018_mean_x'] = Arctic_discharge[['2010_vx','2011_vx','2012_vx','2013_vx','2014_vx','2015_vx','2016_vx','2017_vx','2018_vx']].mean(axis=1,skipna=True)
Arctic_discharge['2010_2018_mean_y'] = Arctic_discharge[['2010_vy','2011_vy','2012_vy','2013_vy','2014_vy','2015_vy','2016_vy','2017_vy','2018_vy']].mean(axis=1,skipna=True)

#Take a mean including other years
Arctic_discharge['2000_2010_mean'] = Arctic_discharge[['2000_v','2001_v','2002_v','2003_v','2004_v','2005_v','2006_v','2007_v','2008_v','2009_v',\
    'Austfonna_ERS_1995','FranzJosefLand_JERS1_1996_1998', 'FranzJosefLand_JERS1_1996_1997',\
    'NovayaZemlya_ALOS1_2008_2010', 'NovayaZemlya_JERS1_1998','SevernayaZemlya_ALOS1_2010',\
    'SevernayaZemlya_ERS1_1991_1992','Svalbard_ALOS1_2007_2011', 'Svalbard_ERS1_1992','Svalbard_JERS1_1994_1998', 'Svalbard_TSX_2008_2012',\
     ]].mean(axis=1,skipna=True)
Arctic_discharge['2000_2010_mean'].fillna(Arctic_discharge['Northern_velocity'], inplace=True)
#fill velocity data for Barents Sea glacier that has no velocity observations
mask = (Arctic_discharge['RGIId']=='RGI60-09.01069')|(Arctic_discharge['RGIId']=='RGI60-10.00004')
Arctic_discharge.loc[mask, '2000_2010_mean']=Arctic_discharge.loc[mask, '2000_2010_mean'].fillna(5)#fill Barents sea and de long glaciers with 5
Arctic_discharge['2000_2010_mean_err'] = Arctic_discharge[['2000_v_err','2001_v_err','2002_v_err','2003_v_err','2004_v_err','2005_v_err','2006_v_err','2007_v_err','2008_v_err','2009_v_err',\
    'Austfonna_ERS_1995_err','FranzJosefLand_JERS1_1996_1998_err', 'FranzJosefLand_JERS1_1996_1997_err',\
    'NovayaZemlya_ALOS1_2008_2010_err', 'NovayaZemlya_JERS1_1998_err','SevernayaZemlya_ALOS1_2010_err',\
    'SevernayaZemlya_ERS1_1991_1992_err','Svalbard_ALOS1_2007_2011_err', 'Svalbard_ERS1_1992_err','Svalbard_JERS1_1994_1998_err', 'Svalbard_TSX_2008_2012_err',\
    ]].mean(axis=1,skipna=True)
Arctic_discharge['2000_2010_mean_err'].fillna(Arctic_discharge['Northern_vel_err'], inplace=True)
Arctic_discharge.loc[mask, '2000_2010_mean_err']=Arctic_discharge.loc[mask, '2000_2010_mean_err'].fillna(5) #fill error with 5 for Barents sea and de long glaciers

Arctic_discharge['2010_2018_mean'] = Arctic_discharge[['2010_v','2011_v','2012_v','2013_v','2014_v','2015_v','2016_v','2017_v','2018_v',\
    'FranzJosefLand_ALOS1_2010_2011','FranzJosefLand_ALOS2_2016']].mean(axis=1,skipna=True)
Arctic_discharge['2010_2018_mean'].fillna(Arctic_discharge['Northern_velocity'], inplace=True)
Arctic_discharge.loc[mask, '2010_2018_mean']=Arctic_discharge.loc[mask, '2010_2018_mean'].fillna(5) #fill Barents sea and de long glaciers with 5
Arctic_discharge['2010_2018_mean_err'] = Arctic_discharge[['2010_v_err','2011_v_err','2012_v_err','2013_v_err','2014_v_err','2015_v_err','2016_v_err','2017_v_err','2018_v_err',\
    'FranzJosefLand_ALOS1_2010_2011_err','FranzJosefLand_ALOS2_2016_err']].mean(axis=1,skipna=True)
Arctic_discharge['2010_2018_mean_err'].fillna(Arctic_discharge['Northern_vel_err'], inplace=True)
Arctic_discharge.loc[mask, '2010_2018_mean_err']=Arctic_discharge.loc[mask, '2010_2018_mean_err'].fillna(5) #fill error with 5 for Barents sea and de long glaciers

# fill gaps in the velocity record
# fill 2000-2010 holes with newer data
Arctic_discharge['2010_2018_mean'] = Arctic_discharge['2010_2018_mean'].interpolate(method='nearest',limit=20) #fill no data gaps with nearby values, limit to prevent data filling in Jan Mayen where there is lots of nans
Arctic_discharge['2010_2018_mean_err'] = Arctic_discharge['2010_2018_mean_err'].interpolate(method='nearest',limit=20) #fill no data gaps with nearby values, limit to prevent data filling in Jan Mayen where there is lots of nans

print('number of nans in 2000-2010 velocity layer that are filled with 2010-2020',Arctic_discharge['2000_2010_mean'].isna().sum(),'out of ', len(Arctic_discharge))
print(Arctic_discharge['2000_2010_mean'].isnull().groupby(Arctic_discharge['region']).sum())

Arctic_discharge['2000_2010_mean'].fillna(Arctic_discharge['2010_2018_mean'], inplace=True)
Arctic_discharge['2000_2010_mean_err'].fillna(Arctic_discharge['2000_2010_mean_err'], inplace=True)

print(np.mean(Arctic_discharge['2000_2010_mean_err']))
#if the error is bigger than the velocity, replace with the velocity value
for i in range(0,len(Arctic_discharge)):
    if Arctic_discharge['2000_2010_mean_err'][i]>Arctic_discharge['2000_2010_mean'][i]:
        Arctic_discharge['2000_2010_mean_err'][i]=Arctic_discharge['2000_2010_mean'][i]
    if Arctic_discharge['2010_2018_mean_err'][i]>Arctic_discharge['2010_2018_mean'][i]:
        Arctic_discharge['2010_2018_mean_err'][i]=Arctic_discharge['2010_2018_mean'][i]
print(np.mean(Arctic_discharge['2000_2010_mean_err']))

/Users/willkochtitzky/opt/miniconda3/envs/old_descartes/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


number of nans in 2000-2010 velocity layer that are filled with 2010-2020 7028 out of  188881
region
Alaska         49.0
Canada          0.0
Greenland       0.0
Iceland       126.0
Russia       4529.0
Svalbard     2324.0
Name: 2000_2010_mean, dtype: float64
122.46192299036618


/Users/willkochtitzky/opt/miniconda3/envs/old_descartes/lib/python3.7/site-packages/ipykernel_launcher.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/willkochtitzky/opt/miniconda3/envs/old_descartes/lib/python3.7/site-packages/ipykernel_launcher.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


46.863417822200155


In [9]:
#figure out glacier thickness
Arctic_discharge['tomo_thickness'].replace(0, np.nan, inplace=True)
Arctic_discharge['tomo_thickness'][Arctic_discharge['tomo_thickness']<0]=np.nan #some large negative values that we delete here
Arctic_discharge['best_thickness']=Arctic_discharge['tomo_thickness']
print('Number of data points filled with tomo data',Arctic_discharge['best_thickness'].count())
Arctic_discharge['best_thickness'].fillna(Arctic_discharge['THICKNESS'], inplace=True) #if tomo is null, then fill with GLATHIDA
print('Number of data points filled with tomo data and GlaThiDa',Arctic_discharge['best_thickness'].count())
Arctic_discharge['best_thickness'].fillna(Arctic_discharge['Russian_thickness'], inplace=True) #if tomo+GlaThiDa are null, then fill with Russian thickness data
print('Number of data points filled with tomo data, GlaThiDa, and Russian data',Arctic_discharge['best_thickness'].count())

#insert uncertainties
# Arctic_discharge=Arctic_discharge.reset_index()
observation_idx=Arctic_discharge['THICKNESS'].notna()|Arctic_discharge['Russian_thickness'].notna()|Arctic_discharge['tomo_thickness'].notna()
Arctic_discharge['thickness_uncertainty']=Arctic_discharge['THICKNESS_UNCERTAINTY']#start by filling with all GlaThiDa data
Arctic_discharge['thickness_uncertainty'][observation_idx]=10

#fill in GlaThiDa uncertainties where you have them
glathida_uncertain_idx=(Arctic_discharge['THICKNESS'].notna())&(Arctic_discharge['THICKNESS_UNCERTAINTY'].notna())
Arctic_discharge['thickness_uncertainty'][glathida_uncertain_idx]=Arctic_discharge['THICKNESS_UNCERTAINTY'][glathida_uncertain_idx]#start by filling with all GlaThiDa data

#calculate data you need for interpolated thickness modeling
#Calculate your own U shaped valley for all glaciers and fill in holes in observations
Arctic_discharge['position']=Arctic_discharge.index #create positions
Arctic_discharge_position_counter = Arctic_discharge[['RGIId','position']].groupby(['RGIId']).min().reset_index()
Arctic_discharge_position_counter = Arctic_discharge_position_counter.rename(columns={"position": "min_position"})
Arctic_discharge=Arctic_discharge.merge(Arctic_discharge_position_counter,how='left',left_on='RGIId',right_on='RGIId')

Arctic_discharge_position_counter = Arctic_discharge[['RGIId','position']].groupby(['RGIId']).max().reset_index()
Arctic_discharge_position_counter = Arctic_discharge_position_counter.rename(columns={"position": "max_position"})
Arctic_discharge=Arctic_discharge.merge(Arctic_discharge_position_counter,how='left',left_on='RGIId',right_on='RGIId')

Arctic_discharge['distance_across']=(Arctic_discharge['position']-Arctic_discharge['min_position'])*25#25 meters for the spacing you defined above
Arctic_discharge['position']=(Arctic_discharge['position']-Arctic_discharge['min_position'])

#create a mask to find areas near the glacier margin
Arctic_discharge['velocity_glacier_edge_mask']=0
for i in range(0,len(Arctic_discharge)):
    if (Arctic_discharge['position'][i]<20)|(Arctic_discharge['max_position'][i]-Arctic_discharge['min_position'][i]-Arctic_discharge['position'][i]<20): #if you are within 500 m (20*25 m points), then we create a mask for velocity orientation mapping later
        Arctic_discharge['velocity_glacier_edge_mask'][i]=1
#drop this to avoid problems later
Arctic_discharge = Arctic_discharge.drop(columns='THICKNESS_UNCERTAINTY')

Number of data points filled with tomo data 8413
Number of data points filled with tomo data and GlaThiDa 16115
Number of data points filled with tomo data, GlaThiDa, and Russian data 30556


/Users/willkochtitzky/opt/miniconda3/envs/old_descartes/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/willkochtitzky/opt/miniconda3/envs/old_descartes/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/Users/willkochtitzky/opt/miniconda3/envs/old_descartes/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

In [10]:
#set up data for interpolated thickness model
glacier_best_thickness=[]
glimsid = []
centerline_distance_across = []

for i in range(0,len(Arctic_discharge)):
    #          dividing by 5 gives you middle 20% of the glacier                             This subtraction is giving you the number of position away you are from the middle, so if you are in the middle 20% these calculations will happen
    if (((Arctic_discharge['max_position'][i]-Arctic_discharge['min_position'][i]))/5 >= np.abs(Arctic_discharge['position'][i]-((Arctic_discharge['max_position'][i]-Arctic_discharge['min_position'][i])/2))): #change 2.5 to include more or less points
        glimsid.append(Arctic_discharge['RGIId'][i])
        glacier_best_thickness.append(Arctic_discharge['best_thickness'][i])
        centerline_distance_across.append(Arctic_discharge['distance_across'][i])

centerline_thickness = pd.DataFrame(data={'RGIId':glimsid,'centerline_depth':glacier_best_thickness,'centerline_distance_across':centerline_distance_across})
centerline_thickness=centerline_thickness.groupby(['RGIId']).mean().reset_index()
centerline_thickness['do_we_have_an_observation_in_middle_fifth']=np.nan
centerline_thickness['do_we_have_an_observation_in_middle_fifth'][centerline_thickness['centerline_depth']>0]=1 #add a one if there is a centerline depth greater than 0 because this means this glacier has a centerline observation
Arctic_discharge=Arctic_discharge.merge(centerline_thickness,how='left',left_on='RGIId',right_on='RGIId')

Arctic_discharge['velocity_glacier_edge_mask']=0
for i in range(0,len(Arctic_discharge)):
    if (Arctic_discharge['position'][i]<20)|(Arctic_discharge['max_position'][i]-Arctic_discharge['min_position'][i]-Arctic_discharge['position'][i]<20): #if you are within 500 m (20*25 m points), then we create a mask for velocity orientation mapping later
        Arctic_discharge['velocity_glacier_edge_mask'][i]=1


#calculate U shaped valley from centerline thickness
centerline_depth=Arctic_discharge['centerline_depth']
distance1=Arctic_discharge['centerline_distance_across']
distance2=np.abs(Arctic_discharge['centerline_distance_across']-Arctic_discharge['distance_across'])
Arctic_discharge['interpolated_thickness']=((10-centerline_depth)/distance1**2)*(distance2**2)+centerline_depth

#figure out where interpolated data will go
interp_idx = Arctic_discharge['best_thickness'].isnull()&(Arctic_discharge['interpolated_thickness']>0)

#fill with interpolated data where possible
Arctic_discharge['best_thickness'].fillna(Arctic_discharge['interpolated_thickness'], inplace=True)

#figure out where Farinotti will go
Farinotti_idx = Arctic_discharge['best_thickness'].isnull()

#fill gaps in best_thickness with Farinotti
Arctic_discharge['best_thickness'].fillna(Arctic_discharge['Farinotti_thickness'], inplace=True) #if still null, fill with Farinotti


#get uncertainties for Farinotti and intepolated
Arctic_discharge['thickness_uncertainty'][interp_idx]=Arctic_discharge['interpolated_thickness']*.10 #10% error on interpolated thickness
Arctic_discharge['thickness_uncertainty'][Farinotti_idx]=Arctic_discharge['Farinotti_thickness']*.26 #26% error on Farinotti thickness

#for Svalbard, just use Farinotti
Arctic_discharge['best_thickness'][Arctic_discharge['region']=='Svalbard']=Arctic_discharge['Farinotti_thickness']
Arctic_discharge['thickness_uncertainty'][Arctic_discharge['region']=='Svalbard']=Arctic_discharge['Farinotti_thickness']*.18#18% error on Svalbard thickness inline with Furst results


/Users/willkochtitzky/opt/miniconda3/envs/old_descartes/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/Users/willkochtitzky/opt/miniconda3/envs/old_descartes/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/willkochtitzky/opt/miniconda3/envs/old_descartes/lib/python3.7/site-packages/ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_gui

In [11]:
Arctic_discharge = Arctic_discharge.sort_values(by=['RGIId','distance_across'])

Arctic_discharge['bearing_to_next_point'] = np.nan
Arctic_discharge['bearing_of_velocity_early'] = np.nan
Arctic_discharge['bearing_of_velocity_late'] = np.nan
Arctic_discharge['velocity_offsett_angle_early'] = np.nan
Arctic_discharge['velocity_offsett_angle_late'] = np.nan
Arctic_discharge['claculated_fluxgate_width_early_min'] = np.nan
Arctic_discharge['claculated_fluxgate_width_late_min'] = np.nan
Arctic_discharge['claculated_fluxgate_width_early_max'] = np.nan
Arctic_discharge['claculated_fluxgate_width_late_max'] = np.nan
Arctic_discharge['length_early_min'] = 25
Arctic_discharge['length_late_min'] = 25
Arctic_discharge['length_early_max'] = 25
Arctic_discharge['length_late_max'] = 25

#calculate the angle that flow is hitting the fluxgate
for i in range(0,len(Arctic_discharge)-1):
    if (Arctic_discharge['RGIId'][i]!=Arctic_discharge['RGIId'][i+1]):
        Arctic_discharge['bearing_to_next_point'][i] = Arctic_discharge['bearing_to_next_point'][i-1]
    elif Arctic_discharge['RGIId'][i]==Arctic_discharge['RGIId'][i+1]:
        hypot=np.sqrt((Arctic_discharge['easting'][i+1]-Arctic_discharge['easting'][i])**2+(Arctic_discharge['northing'][i+1]-Arctic_discharge['northing'][i])**2)
        Arctic_discharge['bearing_to_next_point'][i] = math.atan2((Arctic_discharge['easting'][i+1]-Arctic_discharge['easting'][i])/hypot,(Arctic_discharge['northing'][i+1]-Arctic_discharge['northing'][i])/hypot)*180/math.pi #% 360
        
    Arctic_discharge['bearing_of_velocity_early'][i] = math.atan2(Arctic_discharge['2000_2010_mean_x'][i]/Arctic_discharge['2000_2010_mean'][i],Arctic_discharge['2000_2010_mean_y'][i]/Arctic_discharge['2000_2010_mean'][i])*180/math.pi+45 % 360 #need +45 to convert coordiante systems
    Arctic_discharge['bearing_of_velocity_late'][i] = math.atan2(Arctic_discharge['2010_2018_mean_x'][i]/Arctic_discharge['2010_2018_mean'][i],Arctic_discharge['2010_2018_mean_y'][i]/Arctic_discharge['2010_2018_mean'][i])*180/math.pi+45 % 360 #need +45 to convert coordiante systems
    
    Arctic_discharge['velocity_offsett_angle_early'][i] = np.abs((Arctic_discharge['bearing_to_next_point'][i] - Arctic_discharge['bearing_of_velocity_early'][i]) % 360)
    if Arctic_discharge['velocity_offsett_angle_early'][i]>180:
        Arctic_discharge['velocity_offsett_angle_early'][i]=360-Arctic_discharge['velocity_offsett_angle_early'][i]
    if Arctic_discharge['velocity_offsett_angle_early'][i]>90:
        Arctic_discharge['claculated_fluxgate_width_early_min'][i] = np.abs(math.sin(math.radians(Arctic_discharge['velocity_offsett_angle_early'][i]+10))*25)
        Arctic_discharge['claculated_fluxgate_width_early_max'][i] = np.abs(math.sin(math.radians(Arctic_discharge['velocity_offsett_angle_early'][i]-10))*25)
    if Arctic_discharge['velocity_offsett_angle_early'][i]<90:
        Arctic_discharge['claculated_fluxgate_width_early_min'][i] = np.abs(math.sin(math.radians(Arctic_discharge['velocity_offsett_angle_early'][i]-10))*25)
        Arctic_discharge['claculated_fluxgate_width_early_max'][i] = np.abs(math.sin(math.radians(Arctic_discharge['velocity_offsett_angle_early'][i]+10))*25)
    
    Arctic_discharge['velocity_offsett_angle_late'][i] = np.abs((Arctic_discharge['bearing_to_next_point'][i] - Arctic_discharge['bearing_of_velocity_late'][i]) % 360)
    if Arctic_discharge['velocity_offsett_angle_late'][i]>180:
        Arctic_discharge['velocity_offsett_angle_late'][i]=360-Arctic_discharge['velocity_offsett_angle_late'][i]
    if Arctic_discharge['velocity_offsett_angle_late'][i]>90:
        Arctic_discharge['claculated_fluxgate_width_late_min'][i] = np.abs(math.sin(math.radians(Arctic_discharge['velocity_offsett_angle_late'][i]+10))*25)
        Arctic_discharge['claculated_fluxgate_width_late_max'][i] = np.abs(math.sin(math.radians(Arctic_discharge['velocity_offsett_angle_late'][i]-10))*25)
    if Arctic_discharge['velocity_offsett_angle_late'][i]<90:
        Arctic_discharge['claculated_fluxgate_width_late_min'][i] = np.abs(math.sin(math.radians(Arctic_discharge['velocity_offsett_angle_late'][i]-10))*25)
        Arctic_discharge['claculated_fluxgate_width_late_max'][i] = np.abs(math.sin(math.radians(Arctic_discharge['velocity_offsett_angle_late'][i]+10))*25)

    
    #if you have slow velocities, replace fluxgate with 25 m width
    if (Arctic_discharge['2000_2010_mean'][i] > 25)&(45<Arctic_discharge['velocity_offsett_angle_early'][i]<135):
        Arctic_discharge['length_early_min'][i] = Arctic_discharge['claculated_fluxgate_width_early_min'][i]
        Arctic_discharge['length_early_max'][i] = Arctic_discharge['claculated_fluxgate_width_early_max'][i]
    elif (Arctic_discharge['2000_2010_mean'][i] < 25)|(Arctic_discharge['velocity_offsett_angle_early'][i]<45)|(Arctic_discharge['velocity_offsett_angle_early'][i]>135)|(Arctic_discharge['velocity_glacier_edge_mask'][i] == 1):#if the velocity is either slow, high angle, or near the edge, replace with max uncertainty
        Arctic_discharge['length_early_min'][i] = np.abs(math.sin(math.radians(45))*25)#gate width at 45°
        Arctic_discharge['length_early_max'][i] = 25        
        
    if (Arctic_discharge['2010_2018_mean'][i] > 25)&(45<Arctic_discharge['velocity_offsett_angle_late'][i]<135):
        Arctic_discharge['length_late_min'][i] = Arctic_discharge['claculated_fluxgate_width_late_min'][i]
        Arctic_discharge['length_late_max'][i] = Arctic_discharge['claculated_fluxgate_width_late_max'][i]
    elif (Arctic_discharge['2010_2018_mean'][i] < 25)|(Arctic_discharge['velocity_offsett_angle_late'][i]<45)|(Arctic_discharge['velocity_offsett_angle_late'][i]>135)|(Arctic_discharge['velocity_glacier_edge_mask'][i] == 1):#if the velocity is either slow, high angle, or near the edge, replace with max uncertainty
        Arctic_discharge['length_late_min'][i] = np.abs(math.sin(math.radians(45))*25)#gate width at 45°
        Arctic_discharge['length_late_max'][i] = 25


# Arctic_discharge_testing_export = Arctic_discharge#[2000:8000]
# Arctic_discharge_testing_export = gpd.GeoDataFrame(Arctic_discharge_testing_export, geometry=gpd.points_from_xy(Arctic_discharge_testing_export.easting, Arctic_discharge_testing_export.northing),crs='EPSG:3995')

# Arctic_discharge_testing_export.to_file('Arctic_testing.gpkg', driver="GPKG")

/Users/willkochtitzky/opt/miniconda3/envs/old_descartes/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/willkochtitzky/opt/miniconda3/envs/old_descartes/lib/python3.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/willkochtitzky/opt/miniconda3/envs/old_descartes/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a

In [12]:
#calculate the flux from discharge
#composite data
Arctic_discharge['flux_min'] = (Arctic_discharge['0000_v']-Arctic_discharge['0000_v_err'])*Arctic_discharge['length_late_min']*(Arctic_discharge['best_thickness']-Arctic_discharge['thickness_uncertainty'])*0.9/1000000000
Arctic_discharge['flux_max'] = (Arctic_discharge['0000_v']+Arctic_discharge['0000_v_err'])*Arctic_discharge['length_late_max']*(Arctic_discharge['best_thickness']+Arctic_discharge['thickness_uncertainty'])*1/1000000000

#2000 decadal mean
Arctic_discharge['2000_2010_mean_flux_min'] = (Arctic_discharge['2000_2010_mean']-Arctic_discharge['2000_2010_mean_err'])*Arctic_discharge['length_early_min']*(Arctic_discharge['best_thickness']-Arctic_discharge['thickness_uncertainty'])*0.9/1000000000
Arctic_discharge['2000_2010_mean_flux_max'] = (Arctic_discharge['2000_2010_mean']+Arctic_discharge['2000_2010_mean_err'])*Arctic_discharge['length_early_max']*(Arctic_discharge['best_thickness']+Arctic_discharge['thickness_uncertainty'])*1/1000000000
#2010 decadal mean
Arctic_discharge['2010_2018_mean_flux_min'] = (Arctic_discharge['2010_2018_mean']-Arctic_discharge['2010_2018_mean_err'])*Arctic_discharge['length_late_min']*(Arctic_discharge['best_thickness']-Arctic_discharge['thickness_uncertainty'])*0.9/1000000000
Arctic_discharge['2010_2018_mean_flux_max'] = (Arctic_discharge['2010_2018_mean']+Arctic_discharge['2010_2018_mean_err'])*Arctic_discharge['length_late_max']*(Arctic_discharge['best_thickness']+Arctic_discharge['thickness_uncertainty'])*1/1000000000

Arctic_discharge['flux_min'] = Arctic_discharge['flux_min'].astype(float)
Arctic_discharge['flux_max'] = Arctic_discharge['flux_max'].astype(float)
Arctic_discharge['2000_2010_mean_flux_min'] = Arctic_discharge['2000_2010_mean_flux_min'].astype(float)
Arctic_discharge['2000_2010_mean_flux_max'] = Arctic_discharge['2000_2010_mean_flux_max'].astype(float)
Arctic_discharge['2010_2018_mean_flux_min'] = Arctic_discharge['2010_2018_mean_flux_min'].astype(float)
Arctic_discharge['2010_2018_mean_flux_max'] = Arctic_discharge['2010_2018_mean_flux_max'].astype(float)
Arctic_discharge['thickness_uncertainty'] = Arctic_discharge['thickness_uncertainty'].astype(float)


In [13]:
# write out file for observations for every point
Arctic_discharge.to_file('Arctic_discharge_points_epsg3995_'+date_string+'.gpkg', driver="GPKG")#change file name based on distance

In [14]:
#make it a pandas data frame so we can groupby
Arctic_discharge_df=pd.DataFrame(Arctic_discharge)

#calculate stats
Arctic_discharge_df_sum = Arctic_discharge_df.groupby(['RGIId']).sum().reset_index()
Arctic_discharge_df_min = Arctic_discharge_df.groupby(['RGIId']).min().reset_index()
Arctic_discharge_df_max = Arctic_discharge_df.groupby(['RGIId']).max().reset_index()
Arctic_discharge_df_mean = Arctic_discharge_df.groupby(['RGIId']).mean().reset_index()
Arctic_discharge_df_count = Arctic_discharge_df.groupby(['RGIId']).count().reset_index()

# Clean the flux data
Arctic_discharge_fluxes_sum = Arctic_discharge_df_sum[['RGIId','flux_min','flux_max','2000_2010_mean_flux_min','2000_2010_mean_flux_max', '2010_2018_mean_flux_min', '2010_2018_mean_flux_max']]
Arctic_discharge_fluxes_min = Arctic_discharge_df_min[['RGIId','Name','Area','Zmin', 'Zmax', 'Zmed', 'Slope', 'Aspect', 'Lmax', 'Status',
       'Connect', 'Form', 'TermType', 'Surging', 'Linkages','Farinotti_thickness','region']]
Arctic_discharge_fluxes_max = Arctic_discharge_df_max[['RGIId','Farinotti_thickness','do_we_have_an_observation_in_middle_fifth']]
Arctic_discharge_fluxes_mean = Arctic_discharge_df_mean[['RGIId','easting','northing','Farinotti_thickness','0000_v','best_thickness','thickness_uncertainty','length_early_min','length_early_max','length_late_min','length_late_max','2000_2010_mean','2000_2010_mean_err','2010_2018_mean','2010_2018_mean_err']]
Arctic_discharge_fluxes_count = Arctic_discharge_df_count[['RGIId','CenLon','THICKNESS','tomo_thickness','Russian_thickness']]
Arctic_discharge_fluxes_count = Arctic_discharge_fluxes_count.rename(columns={"CenLon": "num_pts"})
Arctic_discharge_fluxes_count['gate_length_km']=Arctic_discharge_fluxes_count['num_pts']*25/1000 #25 is distance between points you specificed above
data_frames_for_merge = [Arctic_discharge_fluxes_sum, Arctic_discharge_fluxes_min, Arctic_discharge_fluxes_max, Arctic_discharge_fluxes_mean, Arctic_discharge_fluxes_count]

Arctic_discharge_fluxes = reduce(lambda  left,right: pd.merge(left,right,on=['RGIId'],
                                            how='outer'), data_frames_for_merge)
Arctic_discharge_fluxes = Arctic_discharge_fluxes.rename(columns={"Farinotti_thickness_x": "Farinotti_thickness_min",'Farinotti_thickness_y':'Farinotti_thickness_max','Farinotti_thickness':'Farinotti_thickness_mean','THICKNESS':'GlaThiDa_observations','tomo_thickness':'Tomography_observations','Russian_thickness':'Russian_observations','best_thickness':'best_thickness_mean','thickness_uncertainty':'thickness_mean_uncertainty'})  

#calculate average of min and max and error
Arctic_discharge_fluxes['IL_flux_gt'] = (Arctic_discharge_fluxes['flux_min']+Arctic_discharge_fluxes['flux_max'])/2*.900
Arctic_discharge_fluxes['IL_err_gt'] = (Arctic_discharge_fluxes['flux_max']-Arctic_discharge_fluxes['flux_min'])/2*.900

Arctic_discharge_fluxes['2000_2010_mean_flux_gt'] = (Arctic_discharge_fluxes['2000_2010_mean_flux_min']+Arctic_discharge_fluxes['2000_2010_mean_flux_max'])/2*.900
Arctic_discharge_fluxes['2000_2010_mean_err_gt'] = (Arctic_discharge_fluxes['2000_2010_mean_flux_max']-Arctic_discharge_fluxes['2000_2010_mean_flux_min'])/2*.900

Arctic_discharge_fluxes['2010_2018_mean_flux_gt'] = (Arctic_discharge_fluxes['2010_2018_mean_flux_min']+Arctic_discharge_fluxes['2010_2018_mean_flux_max'])/2*.900
Arctic_discharge_fluxes['2010_2018_mean_err_gt'] = (Arctic_discharge_fluxes['2010_2018_mean_flux_max']-Arctic_discharge_fluxes['2010_2018_mean_flux_min'])/2*.900

In [32]:
#calculate the area change of every glacier from 2000 to 2020
#read in data files
termini_2000 = gpd.read_file('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Terminus_positions/termini_2000.geojson')
termini_2010 = gpd.read_file('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Terminus_positions/termini_2010.geojson')
termini_2020 = gpd.read_file('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Terminus_positions/termini_2020.geojson')

termini_2000=termini_2000[termini_2000.geometry.type=='Polygon']
termini_2010=termini_2010[termini_2010.geometry.type=='Polygon']
termini_2020=termini_2020[termini_2020.geometry.type=='Polygon']

termini_2000['geometry'] = termini_2000['geometry'].buffer(0)
termini_2010['geometry'] = termini_2010['geometry'].buffer(0)
termini_2020['geometry'] = termini_2020['geometry'].buffer(0)

termini_2000['time_period']='2000'
termini_2010['time_period']='2010'
termini_2020['time_period']='2020'

termini=pd.concat([termini_2000,termini_2010,termini_2020])

boxes = gpd.read_file('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Terminus_positions/Glacier_area_change_boxes.gpkg')
RGI = gpd.read_file('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Terminus_positions/RGI_Arctic_marine_terminating_glacier_outlines.gpkg')  
RGI['geometry'] = RGI.geometry.buffer(0)
RGI['RGIId']=RGI['RGIid_2']

#Add RGI and satellite image date attributes to the boxes
boxes_attributed = gpd.sjoin(boxes, RGI, how='inner')
boxes_attributed = boxes_attributed.drop(columns=['index_right'])
boxes_attributed = gpd.sjoin(boxes_attributed, termini_2000, how='left')
boxes_attributed = boxes_attributed.drop(columns=['Date_text','time_period','index_right','index','layer'])
boxes_attributed = boxes_attributed.rename(columns={'Landsat_ID':'Early_satellite_id'})
boxes_attributed = gpd.sjoin(boxes_attributed, termini_2010, how='left')
boxes_attributed = boxes_attributed.drop(columns=['Date_text','time_period','index_right','index'])
boxes_attributed = boxes_attributed.rename(columns={'Landsat_ID':'Middle_satellite_id'})
boxes_attributed = gpd.sjoin(boxes_attributed, termini_2020, how='left')
boxes_attributed = boxes_attributed.drop(columns=['Date_text','time_period','index_right','index','path'])
boxes_attributed = boxes_attributed.rename(columns={'Landsat_ID':'Late_satellite_id'})
boxes_attributed['Early_satellite_id'] = boxes_attributed['Early_satellite_id'].astype(str)
boxes_attributed['Middle_satellite_id'] = boxes_attributed['Middle_satellite_id'].astype(str)
boxes_attributed['Late_satellite_id'] = boxes_attributed['Late_satellite_id'].astype(str)
boxes_attributed = boxes_attributed.reset_index()

#get dates for the satellites images for the box attributes
#2000

boxes_attributed['Early_date']=np.nan
for i in range(0,len(boxes_attributed)):
    if boxes_attributed['Early_satellite_id'][i][0]=='L':
        boxes_attributed['Early_date'][i]=pd.to_datetime(boxes_attributed['Early_satellite_id'][i][17:25],format='%Y%m%d')
    if boxes_attributed['Early_satellite_id'][i][0:4]=='AST_':
        boxes_attributed['Early_date'][i]=pd.to_datetime(boxes_attributed['Early_satellite_id'][i][11:19],format='%m%d%Y')
    if boxes_attributed['Early_satellite_id'][i][0:4]=='AST1':
        boxes_attributed['Early_date'][i]=pd.to_datetime(boxes_attributed['Early_satellite_id'][i][12:20],format='%m%d%Y')
    if boxes_attributed['Early_satellite_id'][i][0:1]=='R':
        boxes_attributed['Early_date'][i]=pd.to_datetime(boxes_attributed['Early_satellite_id'][i][18:27],format='%d%m%Y')

#2010
boxes_attributed['Mid_date']=np.nan
for i in range(0,len(boxes_attributed)):
    if boxes_attributed['Middle_satellite_id'][i][0]=='L':
        boxes_attributed['Mid_date'][i]=pd.to_datetime(boxes_attributed['Middle_satellite_id'][i][17:25],format='%Y%m%d')
    if boxes_attributed['Middle_satellite_id'][i][0:4]=='AST_':
        boxes_attributed['Mid_date'][i]=pd.to_datetime(boxes_attributed['Middle_satellite_id'][i][11:19],format='%m%d%Y')
    if boxes_attributed['Middle_satellite_id'][i][0:4]=='AST1':
        boxes_attributed['Mid_date'][i]=pd.to_datetime(boxes_attributed['Middle_satellite_id'][i][12:20],format='%m%d%Y')
    if boxes_attributed['Middle_satellite_id'][i][0:3]=='IMG':
        boxes_attributed['Mid_date'][i]=pd.to_datetime(boxes_attributed['Middle_satellite_id'][i][26:34],format='%d%m%Y')

#2020
boxes_attributed['Late_date']=np.nan
for i in range(0,len(boxes_attributed)):
    if boxes_attributed['Late_satellite_id'][i][0]=='L':
        boxes_attributed['Late_date'][i]=pd.to_datetime(boxes_attributed['Late_satellite_id'][i][17:25],format='%Y%m%d')
    if boxes_attributed['Late_satellite_id'][i][0:4]=='AST_':
        boxes_attributed['Late_date'][i]=pd.to_datetime(boxes_attributed['Late_satellite_id'][i][11:19],format='%m%d%Y')
    if boxes_attributed['Late_satellite_id'][i][0:4]=='AST1':
        boxes_attributed['Late_date'][i]=pd.to_datetime(boxes_attributed['Late_satellite_id'][i][12:20],format='%m%d%Y')


#get only the glacier terminus area within the box
termini_intersect = gpd.overlay(termini,boxes_attributed,how='intersection')#,keep_geom_type=True)
# termini_intersect['area'] = termini_intersect['geometry'].area/1000000

#get the date from the satellite ID - difference for Landsat and ASTER
termini_intersect['Date']=np.nan
for i in range(0,len(termini_intersect)):
    if termini_intersect['Landsat_ID'][i][0]=='L':
        termini_intersect['Date'][i]=pd.to_datetime(termini_intersect['Landsat_ID'][i][17:25],format='%Y%m%d')
    if termini_intersect['Landsat_ID'][i][0:4]=='AST_':
        termini_intersect['Date'][i]=pd.to_datetime(termini_intersect['Landsat_ID'][i][11:19],format='%m%d%Y')
    if termini_intersect['Landsat_ID'][i][0:4]=='AST1':
        termini_intersect['Date'][i]=pd.to_datetime(termini_intersect['Landsat_ID'][i][12:20],format='%m%d%Y')

#sort by terminus date for differencing in next step
early_termini_intersect=termini_intersect[termini_intersect['time_period']=='2000']
middle_termini_intersect=termini_intersect[termini_intersect['time_period']=='2010']
late_termini_intersect=termini_intersect[termini_intersect['time_period']=='2020']

            #2000 to 2020
#calculate total area lost from 2000 to 2020
#different glaciers to get how much area is lost and gained over the time period
marine_based_glacier_area_lost_total = gpd.overlay(early_termini_intersect,late_termini_intersect,how='difference')
marine_based_glacier_area_lost_total=marine_based_glacier_area_lost_total[['RGIId','Name','geometry']].dissolve(by='RGIId')
marine_based_glacier_area_lost_total['area_km2'] = marine_based_glacier_area_lost_total['geometry'].area/1000000
marine_based_glacier_area_lost_total['perimeter'] = marine_based_glacier_area_lost_total['geometry'].length
marine_based_glacier_area_lost_total['perimeter_error'] = marine_based_glacier_area_lost_total['perimeter']/30*30*30/1000000 #/pixels/meter *area/pixel /m2/km2
marine_based_glacier_area_lost_total['perimeter_error'] = marine_based_glacier_area_lost_total['perimeter_error'].fillna(0)
marine_based_glacier_area_lost_total=marine_based_glacier_area_lost_total[marine_based_glacier_area_lost_total['area_km2']>0.000225] #throw out tiny areas that are a mistakes

#calculate total area gained from 2000 to 2020
marine_based_glacier_area_gained_total = gpd.overlay(late_termini_intersect,early_termini_intersect,how='difference')
marine_based_glacier_area_gained_total=marine_based_glacier_area_gained_total[['RGIId','Name','geometry']].dissolve(by='RGIId')
marine_based_glacier_area_gained_total['area_km2'] = marine_based_glacier_area_gained_total['geometry'].area/1000000
marine_based_glacier_area_gained_total['perimeter'] = marine_based_glacier_area_gained_total['geometry'].length
marine_based_glacier_area_gained_total['perimeter_error'] = marine_based_glacier_area_gained_total['perimeter']/30*30*30/1000000 #/pixels/meter *area/pixel /m2/km2
marine_based_glacier_area_gained_total['perimeter_error'] = marine_based_glacier_area_gained_total['perimeter_error'].fillna(0)
marine_based_glacier_area_gained_total=marine_based_glacier_area_gained_total[marine_based_glacier_area_gained_total['area_km2']>0.000225] #throw out tiny areas that are a mistakes

#export data
marine_based_glacier_area_lost_total.to_file('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Terminus_positions/marine_based_glacier_area_lost_total_'+date_string+'.geojson', driver='GeoJSON')
marine_based_glacier_area_gained_total.to_file('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Terminus_positions/marine_based_glacier_area_gained_total_'+date_string+'.geojson', driver='GeoJSON')


            #2000 to 2010
#calculate total area lost from 2000 to 2020
#different glaciers to get how much area is lost and gained over the time period
marine_based_glacier_area_lost_early = gpd.overlay(early_termini_intersect,middle_termini_intersect,how='difference')
marine_based_glacier_area_lost_early=marine_based_glacier_area_lost_early[['RGIId','Name','geometry']].dissolve(by='RGIId')
marine_based_glacier_area_lost_early['area_km2'] = marine_based_glacier_area_lost_early['geometry'].area/1000000
marine_based_glacier_area_lost_early['perimeter'] = marine_based_glacier_area_lost_early['geometry'].length
marine_based_glacier_area_lost_early['perimeter_error'] = marine_based_glacier_area_lost_early['perimeter']/30*30*30/1000000 #/pixels/meter *area/pixel /m2/km2
marine_based_glacier_area_lost_early['perimeter_error'] = marine_based_glacier_area_lost_early['perimeter_error'].fillna(0)
marine_based_glacier_area_lost_early=marine_based_glacier_area_lost_early[marine_based_glacier_area_lost_early['area_km2']>0.000225] #throw out tiny areas that are a mistakes

#calculate total area gained from 2000 to 2020
marine_based_glacier_area_gained_early = gpd.overlay(middle_termini_intersect,early_termini_intersect,how='difference')
marine_based_glacier_area_gained_early=marine_based_glacier_area_gained_early[['RGIId','Name','geometry']].dissolve(by='RGIId')
marine_based_glacier_area_gained_early['area_km2'] = marine_based_glacier_area_gained_early['geometry'].area/1000000
marine_based_glacier_area_gained_early['perimeter'] = marine_based_glacier_area_gained_early['geometry'].length
marine_based_glacier_area_gained_early['perimeter_error'] = marine_based_glacier_area_gained_early['perimeter']/30*30*30/1000000 #/pixels/meter *area/pixel /m2/km2
marine_based_glacier_area_gained_early['perimeter_error'] = marine_based_glacier_area_gained_early['perimeter_error'].fillna(0)
marine_based_glacier_area_gained_early=marine_based_glacier_area_gained_early[marine_based_glacier_area_gained_early['area_km2']>0.000225] #throw out tiny areas that are a mistakes

#export data
marine_based_glacier_area_lost_early.to_file('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Terminus_positions/marine_based_glacier_area_lost_early_'+date_string+'.geojson', driver='GeoJSON')
marine_based_glacier_area_gained_early.to_file('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Terminus_positions/marine_based_glacier_area_gained_early_'+date_string+'.geojson', driver='GeoJSON')


            #2010 to 2020
#calculate total area lost from 2000 to 2020
#different glaciers to get how much area is lost and gained over the time period
marine_based_glacier_area_lost_late = gpd.overlay(middle_termini_intersect,late_termini_intersect,how='difference')
marine_based_glacier_area_lost_late=marine_based_glacier_area_lost_late.reset_index()[['RGIId','Name','geometry']].dissolve(by='RGIId')
marine_based_glacier_area_lost_late['area_km2'] = marine_based_glacier_area_lost_late['geometry'].area/1000000
marine_based_glacier_area_lost_late['perimeter'] = marine_based_glacier_area_lost_late['geometry'].length
marine_based_glacier_area_lost_late['perimeter_error'] = marine_based_glacier_area_lost_late['perimeter']/30*30*30/1000000 #/pixels/meter *area/pixel /m2/km2
marine_based_glacier_area_lost_late['perimeter_error'] = marine_based_glacier_area_lost_late['perimeter_error'].fillna(0)
marine_based_glacier_area_lost_late=marine_based_glacier_area_lost_late[marine_based_glacier_area_lost_late['area_km2']>0.000225] #throw out tiny areas that are a mistakes

#calculate total area gained from 2010 to 2020
marine_based_glacier_area_gained_late = gpd.overlay(late_termini_intersect,middle_termini_intersect,how='difference')
marine_based_glacier_area_gained_late=marine_based_glacier_area_gained_late[['RGIId','Name','geometry']].dissolve(by='RGIId')
marine_based_glacier_area_gained_late['area_km2'] = marine_based_glacier_area_gained_late['geometry'].area/1000000
marine_based_glacier_area_gained_late['perimeter'] = marine_based_glacier_area_gained_late['geometry'].length
marine_based_glacier_area_gained_late['perimeter_error'] = marine_based_glacier_area_gained_late['perimeter']/30*30*30/1000000 #/pixels/meter *area/pixel /m2/km2
marine_based_glacier_area_gained_late['perimeter_error']=marine_based_glacier_area_gained_late['perimeter_error'].fillna(0)
marine_based_glacier_area_gained_late=marine_based_glacier_area_gained_late[marine_based_glacier_area_gained_late['area_km2']>0.000225] #throw out tiny areas that are a mistakes

#export data
marine_based_glacier_area_lost_late.to_file('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Terminus_positions/marine_based_glacier_area_lost_late_'+date_string+'.geojson', driver='GeoJSON')
marine_based_glacier_area_gained_late.to_file('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Terminus_positions/marine_based_glacier_area_gained_late_'+date_string+'.geojson', driver='GeoJSON')


#create one list of all the glaciers we have terminus area change for
all_glacier_list=marine_based_glacier_area_lost_early.append([marine_based_glacier_area_gained_early, marine_based_glacier_area_lost_late, marine_based_glacier_area_gained_late])
all_glacier_list=all_glacier_list.groupby('RGIId').min().reset_index()[['RGIId','Name']]

#merge datasets to calculate net area change
all_glacier_list=pd.merge(all_glacier_list,marine_based_glacier_area_gained_total,on=['RGIId'],how='left')
net_area_change = pd.merge(all_glacier_list,marine_based_glacier_area_lost_total,on=['RGIId'],how='left').drop(columns=['Name_y','Name']).rename(columns={"geometry_x": "geometry_gained_total", "area_km2_x": "area_km2_gained_total","geometry_y": "geometry_lost_total", "area_km2_y": "area_km2_lost_total",'Name_x':'Name','perimeter_x':'perimeter_gained_total','perimeter_y':'perimeter_lost_total','perimeter_error_x':'perimeter_error_gained_total','perimeter_error_y':'perimeter_error_lost_total'})
net_area_change = pd.merge(net_area_change,marine_based_glacier_area_lost_early,on=['RGIId'],how='left').drop(columns=['Name_y']).rename(columns={"geometry": "geometry_lost_early", "area_km2": "area_km2_lost_early",'Name_x':'Name','perimeter':'perimeter_lost_early','perimeter_error':'perimeter_error_lost_early'})
net_area_change = pd.merge(net_area_change,marine_based_glacier_area_gained_early,on=['RGIId'],how='left').drop(columns=['Name_y']).rename(columns={"geometry": "geometry_gained_early", "area_km2": "area_km2_gained_early",'Name_x':'Name','perimeter':'perimeter_gained_early','perimeter_error':'perimeter_error_gained_early'})
net_area_change = pd.merge(net_area_change,marine_based_glacier_area_lost_late,on=['RGIId'],how='left').drop(columns=['Name_y']).rename(columns={"geometry": "geometry_lost_late", "area_km2": "area_km2_lost_late",'Name_x':'Name','perimeter':'perimeter_lost_late','perimeter_error':'perimeter_error_lost_late'})
net_area_change = pd.merge(net_area_change,marine_based_glacier_area_gained_late,on=['RGIId'],how='left').drop(columns=['Name_y']).rename(columns={"geometry": "geometry_gained_late", "area_km2": "area_km2_gained_late",'Name_x':'Name','perimeter':'perimeter_gained_late','perimeter_error':'perimeter_error_gained_late'})
net_area_change = pd.merge(net_area_change,boxes_attributed[['RGIId','Early_satellite_id','Middle_satellite_id', 'Late_satellite_id', 'Early_date','Mid_date','Late_date']],on=['RGIId'],how='left')


#Set areas to 0 where there is no gain (but a loss) or no loss (but a gain) so that you still get results for net change 
net_area_change=net_area_change.reset_index()
for i in range(0,len(net_area_change)):
    if net_area_change['area_km2_gained_early'].isnull()[i] and (net_area_change['area_km2_lost_early'][i]>0):
        net_area_change['area_km2_gained_early'][i]=0
    if net_area_change['area_km2_lost_early'].isnull()[i] and (net_area_change['area_km2_gained_early'][i]>0):
        net_area_change['area_km2_lost_early'][i]=0
        
    if net_area_change['area_km2_gained_late'].isnull()[i] and (net_area_change['area_km2_lost_late'][i]>0):
        net_area_change['area_km2_gained_late'][i]=0
    if net_area_change['area_km2_lost_late'].isnull()[i] and (net_area_change['area_km2_gained_late'][i]>0):
        net_area_change['area_km2_lost_late'][i]=0
        
    if net_area_change['area_km2_gained_total'].isnull()[i] and (net_area_change['area_km2_lost_total'][i]>0):
        net_area_change['area_km2_gained_total'][i]=0
    if net_area_change['area_km2_lost_total'].isnull()[i] and (net_area_change['area_km2_gained_total'][i]>0):
        net_area_change['area_km2_lost_total'][i]=0


#calculate time elapsed between two images
net_area_change['Early_date_decimal_year']=np.nan
net_area_change['Middle_date_decimal_year']=np.nan
net_area_change['Late_date_decimal_year']=np.nan
for i in range(0,len(net_area_change)):
    if pd.isna(net_area_change['Early_date'][i])==False:#this way it skips nan values
        net_area_change['Early_date_decimal_year'][i] = net_area_change['Early_date'][i].year+net_area_change['Early_date'][i].dayofyear/365
    if pd.isna(net_area_change['Mid_date'][i])==False:#this way it skips nan values
        net_area_change['Middle_date_decimal_year'][i] = net_area_change['Mid_date'][i].year+net_area_change['Mid_date'][i].dayofyear/365
    if pd.isna(net_area_change['Late_date'][i])==False:#this way it skips nan values
        net_area_change['Late_date_decimal_year'][i] = net_area_change['Late_date'][i].year+net_area_change['Late_date'][i].dayofyear/365
net_area_change['total_years_elapsed']=net_area_change['Late_date_decimal_year']-net_area_change['Early_date_decimal_year']
net_area_change['early_to_middle_years_elapsed']=net_area_change['Middle_date_decimal_year']-net_area_change['Early_date_decimal_year']
net_area_change['middle_to_late_years_elapsed']=net_area_change['Late_date_decimal_year']-net_area_change['Middle_date_decimal_year']

net_area_change['total_years_elapsed'].fillna(10,inplace=True)
net_area_change['early_to_middle_years_elapsed'].fillna(10,inplace=True)
net_area_change['middle_to_late_years_elapsed'].fillna(10,inplace=True)

#clean up the dataset by replacing nans with 0 (since you have no area if nan) and then find net change
net_area_change=net_area_change.fillna(value={'area_km2_gained_early':0,'area_km2_gained_late':0,'area_km2_lost_early':0,'area_km2_lost_late':0,'perimeter_error_gained_early':0,'perimeter_error_lost_early':0,'perimeter_error_gained_late':0,'perimeter_error_lost_late':0})
net_area_change=net_area_change.drop_duplicates(subset='RGIId')
net_area_change['net_area_change_total']=net_area_change['area_km2_gained_total']-net_area_change['area_km2_lost_total']
net_area_change['net_area_change_total_error']=net_area_change['perimeter_error_gained_total']+net_area_change['perimeter_error_lost_total']
net_area_change['net_area_change_per_year_total']=net_area_change['net_area_change_total']/net_area_change['total_years_elapsed']
net_area_change['net_area_change_per_year_total_error']=net_area_change['net_area_change_total_error']/net_area_change['total_years_elapsed']
net_area_change['net_area_change_early']=net_area_change['area_km2_gained_early']-net_area_change['area_km2_lost_early']
net_area_change['net_area_change_early_error']=net_area_change['perimeter_error_gained_early']+net_area_change['perimeter_error_lost_early']
net_area_change['net_area_change_per_year_early']=net_area_change['net_area_change_early']/net_area_change['early_to_middle_years_elapsed']
net_area_change['net_area_change_per_year_early_error']=net_area_change['net_area_change_early_error']/net_area_change['early_to_middle_years_elapsed']
net_area_change['net_area_change_late']=net_area_change['area_km2_gained_late']-net_area_change['area_km2_lost_late']
net_area_change['net_area_change_late_error']=net_area_change['perimeter_error_gained_late']+net_area_change['perimeter_error_lost_late']
net_area_change['net_area_change_per_year_late']=net_area_change['net_area_change_late']/net_area_change['middle_to_late_years_elapsed']
net_area_change['net_area_change_per_year_late_error']=net_area_change['net_area_change_late_error']/net_area_change['middle_to_late_years_elapsed']

#make pretty dates
net_area_change['Early_date'] = net_area_change['Early_date'].dt.strftime('%Y/%m/%d')
net_area_change['Mid_date'] = net_area_change['Mid_date'].dt.strftime('%Y/%m/%d')
net_area_change['Late_date'] = net_area_change['Late_date'].dt.strftime('%Y/%m/%d')

#Do not know why I need this, but you do to make the total area change numbers work out
net_area_change['net_area_change_total_error']=net_area_change['perimeter_error_gained_total']+net_area_change['perimeter_error_lost_total']
net_area_change['net_area_change_per_year_total_error']=net_area_change['net_area_change_total_error']/net_area_change['total_years_elapsed']

/Users/willkochtitzky/opt/miniconda3/envs/old_descartes/lib/python3.7/site-packages/ipykernel_launcher.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/willkochtitzky/opt/miniconda3/envs/old_descartes/lib/python3.7/site-packages/ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/willkochtitzky/opt/miniconda3/envs/old_descartes/lib/python3.7/site-packages/ipykernel_launcher.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a

In [36]:
net_area_change.sum()[0:60]

index                                                                             1372220
RGIId                                   RGI60-01.03377RGI60-01.03622RGI60-01.03890RGI6...
area_km2_gained_total                                                             353.435
perimeter_gained_total                                                             768934
perimeter_error_gained_total                                                       23.068
area_km2_lost_total                                                                  3033
perimeter_lost_total                                                          1.19851e+07
perimeter_error_lost_total                                                        359.553
area_km2_lost_early                                                               1556.18
perimeter_lost_early                                                          1.11004e+07
perimeter_error_lost_early                                                        333.011
area_km2_g

In [37]:
net_area_change.dtypes[0:60]

index                                      int64
RGIId                                     object
Name                                      object
geometry_gained_total                   geometry
area_km2_gained_total                    float64
perimeter_gained_total                   float64
perimeter_error_gained_total             float64
geometry_lost_total                     geometry
area_km2_lost_total                      float64
perimeter_lost_total                     float64
perimeter_error_lost_total               float64
geometry_lost_early                     geometry
area_km2_lost_early                      float64
perimeter_lost_early                     float64
perimeter_error_lost_early               float64
geometry_gained_early                   geometry
area_km2_gained_early                    float64
perimeter_gained_early                   float64
perimeter_error_gained_early             float64
geometry_lost_late                      geometry
area_km2_lost_late  

In [38]:
#Fill zeros in area change data so that errors dont come out null
net_area_change[['index', 'RGIId', 'Name', #'geometry_gained_total',
       'area_km2_gained_total', 'perimeter_gained_total',
       'perimeter_error_gained_total', #'geometry_lost_total',
       'area_km2_lost_total', 'perimeter_lost_total',
       'perimeter_error_lost_total', #'geometry_lost_early',
       'area_km2_lost_early', 'perimeter_lost_early',
       'perimeter_error_lost_early', #'geometry_gained_early',
       'area_km2_gained_early', 'perimeter_gained_early',
       'perimeter_error_gained_early', #'geometry_lost_late',
       'area_km2_lost_late', 'perimeter_lost_late',
       'perimeter_error_lost_late', #'geometry_gained_late',
       'area_km2_gained_late', 'perimeter_gained_late',
       'perimeter_error_gained_late', 'Early_date', 'Mid_date',
       'Late_date', 'Early_date_decimal_year', 'Middle_date_decimal_year',
       'Late_date_decimal_year', 'total_years_elapsed',
       'early_to_middle_years_elapsed', 'middle_to_late_years_elapsed',
       'net_area_change_total', 'net_area_change_total_error',
       'net_area_change_per_year_total',
       'net_area_change_per_year_total_error', 'net_area_change_early',
       'net_area_change_early_error', 'net_area_change_per_year_early',
       'net_area_change_per_year_early_error', 'net_area_change_late',
       'net_area_change_late_error', 'net_area_change_per_year_late',
       'net_area_change_per_year_late_error']] = net_area_change[['index', 'RGIId', 'Name', #'geometry_gained_total',
       'area_km2_gained_total', 'perimeter_gained_total',
       'perimeter_error_gained_total', #'geometry_lost_total',
       'area_km2_lost_total', 'perimeter_lost_total',
       'perimeter_error_lost_total', #'geometry_lost_early',
       'area_km2_lost_early', 'perimeter_lost_early',
       'perimeter_error_lost_early', #'geometry_gained_early',
       'area_km2_gained_early', 'perimeter_gained_early',
       'perimeter_error_gained_early',# 'geometry_lost_late',
       'area_km2_lost_late', 'perimeter_lost_late',
       'perimeter_error_lost_late',# 'geometry_gained_late',
       'area_km2_gained_late', 'perimeter_gained_late',
       'perimeter_error_gained_late', 'Early_date', 'Mid_date',
       'Late_date', 'Early_date_decimal_year', 'Middle_date_decimal_year',
       'Late_date_decimal_year', 'total_years_elapsed',
       'early_to_middle_years_elapsed', 'middle_to_late_years_elapsed',
       'net_area_change_total', 'net_area_change_total_error',
       'net_area_change_per_year_total',
       'net_area_change_per_year_total_error', 'net_area_change_early',
       'net_area_change_early_error', 'net_area_change_per_year_early',
       'net_area_change_per_year_early_error', 'net_area_change_late',
       'net_area_change_late_error', 'net_area_change_per_year_late',
       'net_area_change_per_year_late_error']].fillna(value=0)

In [40]:
net_area_change = net_area_change.reset_index()
column_names = ['geometry_gained_total','geometry_lost_total','geometry_lost_early','geometry_gained_early','geometry_lost_late','geometry_gained_late']
for c in column_names:
    for i in range(0,len(net_area_change)):
        try:
            if net_area_change[c][i] == None:
                net_area_change[c][i] = Polygon([(0, 0),(0, 0),(0, 0)])
        except:
              print("An exception occurred")

In [42]:
# !pip show pandas

In [43]:
# !pip install Pandas==0.25.1

In [44]:
# net_area_change = net_area_change.reset_index()

In [45]:
#Figure out thickness of lost/gained glacier area

GlaThiDa= gpd.read_file('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Arctic_GlaThiDa_EPSG3995.gpkg')
Russia_thickness_andrey = gpd.read_file('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/Russia/Russia_thickness_merged_EPSG3995.geojson')
Russia_thickness_julian = gpd.read_file('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/RES_data/RES_ice_thickness.gpkg')

net_area_change['average_GlaThiDa_thickness_lost_early'] = np.nan
net_area_change['average_GlaThiDa_thickness_gained_early'] = np.nan
net_area_change['average_GlaThiDa_thickness_lost_late'] = np.nan
net_area_change['average_GlaThiDa_thickness_gained_late'] = np.nan
net_area_change['average_RUS_andrey_thickness_lost_early'] = np.nan
net_area_change['average_RUS_andrey_thickness_gained_early'] = np.nan
net_area_change['average_RUS_andrey_thickness_lost_late'] = np.nan
net_area_change['average_RUS_andrey_thickness_gained_late'] = np.nan
net_area_change['average_RUS_julian_thickness_lost_early'] = np.nan
net_area_change['average_RUS_julian_thickness_gained_early'] = np.nan
net_area_change['average_RUS_julian_thickness_lost_late'] = np.nan
net_area_change['average_RUS_julian_thickness_gained_late'] = np.nan

for i in range(0,len(net_area_change)):
    if net_area_change['geometry_lost_early'][i]!=0:
        net_area_change['average_GlaThiDa_thickness_lost_early'][i] = GlaThiDa[GlaThiDa.within(net_area_change['geometry_lost_early'][i])]['THICKNESS'].mean()
        net_area_change['average_RUS_andrey_thickness_lost_early'][i] = Russia_thickness_andrey[Russia_thickness_andrey.within(net_area_change['geometry_lost_early'][i])]['Russian_thickness'].mean()
        net_area_change['average_RUS_julian_thickness_lost_early'][i] = Russia_thickness_julian[Russia_thickness_julian.within(net_area_change['geometry_lost_early'][i])]['thickness(m)'].mean()
    if net_area_change['geometry_gained_early'][i]!=0:
        net_area_change['average_GlaThiDa_thickness_gained_early'][i] = GlaThiDa[GlaThiDa.within(net_area_change['geometry_gained_early'][i])]['THICKNESS'].mean()
        net_area_change['average_RUS_andrey_thickness_gained_early'][i] = Russia_thickness_andrey[Russia_thickness_andrey.within(net_area_change['geometry_gained_early'][i])]['Russian_thickness'].mean()
        net_area_change['average_RUS_julian_thickness_gained_early'][i] = Russia_thickness_julian[Russia_thickness_julian.within(net_area_change['geometry_gained_early'][i])]['thickness(m)'].mean()
    if net_area_change['geometry_lost_late'][i]!=0:
        net_area_change['average_GlaThiDa_thickness_lost_late'][i] = GlaThiDa[GlaThiDa.within(net_area_change['geometry_lost_late'][i])]['THICKNESS'].mean()
        net_area_change['average_RUS_andrey_thickness_lost_late'][i] = Russia_thickness_andrey[Russia_thickness_andrey.within(net_area_change['geometry_lost_late'][i])]['Russian_thickness'].mean()
        net_area_change['average_RUS_julian_thickness_lost_late'][i] = Russia_thickness_julian[Russia_thickness_julian.within(net_area_change['geometry_lost_late'][i])]['thickness(m)'].mean()
    if net_area_change['geometry_gained_late'][i]!=0:
        net_area_change['average_GlaThiDa_thickness_gained_late'][i] = GlaThiDa[GlaThiDa.within(net_area_change['geometry_gained_late'][i])]['THICKNESS'].mean()
        net_area_change['average_RUS_andrey_thickness_gained_late'][i] = Russia_thickness_andrey[Russia_thickness_andrey.within(net_area_change['geometry_gained_late'][i])]['Russian_thickness'].mean()
        net_area_change['average_RUS_julian_thickness_gained_late'][i] = Russia_thickness_julian[Russia_thickness_julian.within(net_area_change['geometry_gained_late'][i])]['thickness(m)'].mean()

#Find the Glacier thickness for the areas lost/gained from the Farinotti and other raster datasets
net_area_change['average_Farinotti_thickness_lost_early'] = np.nan
net_area_change['average_Farinotti_thickness_gained_early'] = np.nan
net_area_change['average_Farinotti_thickness_lost_late'] = np.nan
net_area_change['average_Farinotti_thickness_gained_late'] = np.nan

net_area_change['average_tomo_thickness_lost_early'] = np.nan
net_area_change['average_tomo_thickness_gained_early'] = np.nan
net_area_change['average_tomo_thickness_lost_late'] = np.nan
net_area_change['average_tomo_thickness_gained_late'] = np.nan

for i in range(0,len(net_area_change)):
    #ALASKA
    if net_area_change['RGIId'][i][7]=='1':
        if net_area_change['geometry_lost_early'][i]!=0:
            net_area_change['average_Farinotti_thickness_lost_early'][i]=rasterstats.zonal_stats(net_area_change['geometry_lost_early'][i],'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/RGI60-01/aa_RGI6_thickness_mosaic.vrt',stats='mean')[0]['mean']
        if net_area_change['geometry_gained_early'][i]!=0:
            net_area_change['average_Farinotti_thickness_gained_early'][i]=rasterstats.zonal_stats(net_area_change['geometry_gained_early'][i],'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/RGI60-01/aa_RGI6_thickness_mosaic.vrt',stats='mean')[0]['mean']
        if net_area_change['geometry_lost_late'][i]!=0:
            net_area_change['average_Farinotti_thickness_lost_late'][i]=rasterstats.zonal_stats(net_area_change['geometry_lost_late'][i],'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/RGI60-01/aa_RGI6_thickness_mosaic.vrt',stats='mean')[0]['mean']
        if net_area_change['geometry_gained_late'][i]!=0:
            net_area_change['average_Farinotti_thickness_gained_late'][i]=rasterstats.zonal_stats(net_area_change['geometry_gained_late'][i],'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/RGI60-01/aa_RGI6_thickness_mosaic.vrt',stats='mean')[0]['mean']
    #QEI
    if net_area_change['RGIId'][i][7]=='3':
        if net_area_change['geometry_lost_early'][i]!=0:
            net_area_change['average_Farinotti_thickness_lost_early'][i]=rasterstats.zonal_stats(net_area_change['geometry_lost_early'][i],'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/RGI60-03/aa_RGI6_thickness_mosaic.vrt',stats='mean')[0]['mean']
            net_area_change['average_tomo_thickness_lost_early'][i]=rasterstats.zonal_stats(net_area_change['geometry_lost_early'][i],'/Users/willkochtitzky/Projects/CAA_Velocity_Update_from_Wes/3995_polar_stereographic_all_data/tomo_glacier_thickness_0_nodata2_v5.tif',nodata=0,stats='mean')[0]['mean']
        if net_area_change['geometry_gained_early'][i]!=0:
            net_area_change['average_Farinotti_thickness_gained_early'][i]=rasterstats.zonal_stats(net_area_change['geometry_gained_early'][i],'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/RGI60-03/aa_RGI6_thickness_mosaic.vrt',stats='mean')[0]['mean']
            net_area_change['average_tomo_thickness_gained_early'][i]=rasterstats.zonal_stats(net_area_change['geometry_gained_early'][i],'/Users/willkochtitzky/Projects/CAA_Velocity_Update_from_Wes/3995_polar_stereographic_all_data/tomo_glacier_thickness_0_nodata2_v5.tif',nodata=0,stats='mean')[0]['mean']
        if net_area_change['geometry_lost_late'][i]!=0:
            net_area_change['average_Farinotti_thickness_lost_late'][i]=rasterstats.zonal_stats(net_area_change['geometry_lost_late'][i],'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/RGI60-03/aa_RGI6_thickness_mosaic.vrt',stats='mean')[0]['mean']
            net_area_change['average_tomo_thickness_lost_late'][i]=rasterstats.zonal_stats(net_area_change['geometry_lost_late'][i],'/Users/willkochtitzky/Projects/CAA_Velocity_Update_from_Wes/3995_polar_stereographic_all_data/tomo_glacier_thickness_0_nodata2_v5.tif',nodata=0,stats='mean')[0]['mean']
        if net_area_change['geometry_gained_late'][i]!=0:
            net_area_change['average_Farinotti_thickness_gained_late'][i]=rasterstats.zonal_stats(net_area_change['geometry_gained_late'][i],'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/RGI60-03/aa_RGI6_thickness_mosaic.vrt',stats='mean')[0]['mean']
            net_area_change['average_tomo_thickness_gained_late'][i]=rasterstats.zonal_stats(net_area_change['geometry_gained_late'][i],'/Users/willkochtitzky/Projects/CAA_Velocity_Update_from_Wes/3995_polar_stereographic_all_data/tomo_glacier_thickness_0_nodata2_v5.tif',nodata=0,stats='mean')[0]['mean']
    #BBI
    if net_area_change['RGIId'][i][7]=='4':
        if net_area_change['geometry_lost_early'][i]!=0:
            net_area_change['average_Farinotti_thickness_lost_early'][i]=rasterstats.zonal_stats(net_area_change['geometry_lost_early'][i],'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/RGI60-04/aa_RGI6_thickness_mosaic.vrt',stats='mean')[0]['mean']
        if net_area_change['geometry_gained_early'][i]!=0:
            net_area_change['average_Farinotti_thickness_gained_early'][i]=rasterstats.zonal_stats(net_area_change['geometry_gained_early'][i],'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/RGI60-04/aa_RGI6_thickness_mosaic.vrt',stats='mean')[0]['mean']
        if net_area_change['geometry_lost_late'][i]!=0:
            net_area_change['average_Farinotti_thickness_lost_late'][i]=rasterstats.zonal_stats(net_area_change['geometry_lost_late'][i],'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/RGI60-04/aa_RGI6_thickness_mosaic.vrt',stats='mean')[0]['mean']
        if net_area_change['geometry_gained_late'][i]!=0:
            net_area_change['average_Farinotti_thickness_gained_late'][i]=rasterstats.zonal_stats(net_area_change['geometry_gained_late'][i],'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/RGI60-04/aa_RGI6_thickness_mosaic.vrt',stats='mean')[0]['mean']
    #Greenland
    if net_area_change['RGIId'][i][7]=='5':
        if net_area_change['geometry_lost_early'][i]!=0:
            net_area_change['average_Farinotti_thickness_lost_early'][i]=rasterstats.zonal_stats(net_area_change['geometry_lost_early'][i],'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/RGI60-05/aa_RGI6_thickness_mosaic.vrt',stats='mean')[0]['mean']
        if net_area_change['geometry_gained_early'][i]!=0:
            net_area_change['average_Farinotti_thickness_gained_early'][i]=rasterstats.zonal_stats(net_area_change['geometry_gained_early'][i],'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/RGI60-05/aa_RGI6_thickness_mosaic.vrt',stats='mean')[0]['mean']
        if net_area_change['geometry_lost_late'][i]!=0:
            net_area_change['average_Farinotti_thickness_lost_late'][i]=rasterstats.zonal_stats(net_area_change['geometry_lost_late'][i],'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/RGI60-05/aa_RGI6_thickness_mosaic.vrt',stats='mean')[0]['mean']
        if net_area_change['geometry_gained_late'][i]!=0:
            net_area_change['average_Farinotti_thickness_gained_late'][i]=rasterstats.zonal_stats(net_area_change['geometry_gained_late'][i],'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/RGI60-05/aa_RGI6_thickness_mosaic.vrt',stats='mean')[0]['mean']
    #Iceland
    if net_area_change['RGIId'][i][7]=='6':
        if net_area_change['geometry_lost_early'][i]!=0:
            net_area_change['average_Farinotti_thickness_lost_early'][i]=rasterstats.zonal_stats(net_area_change['geometry_lost_early'][i],'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/RGI60-06/aa_RGI6_thickness_mosaic.vrt',stats='mean')[0]['mean']
        if net_area_change['geometry_gained_early'][i]!=0:
            net_area_change['average_Farinotti_thickness_gained_early'][i]=rasterstats.zonal_stats(net_area_change['geometry_gained_early'][i],'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/RGI60-06/aa_RGI6_thickness_mosaic.vrt',stats='mean')[0]['mean']
        if net_area_change['geometry_lost_late'][i]!=0:
            net_area_change['average_Farinotti_thickness_lost_late'][i]=rasterstats.zonal_stats(net_area_change['geometry_lost_late'][i],'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/RGI60-06/aa_RGI6_thickness_mosaic.vrt',stats='mean')[0]['mean']
        if net_area_change['geometry_gained_late'][i]!=0:
            net_area_change['average_Farinotti_thickness_gained_late'][i]=rasterstats.zonal_stats(net_area_change['geometry_gained_late'][i],'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/RGI60-06/aa_RGI6_thickness_mosaic.vrt',stats='mean')[0]['mean']
    #Svalbard
    if net_area_change['RGIId'][i][7]=='7':
        if net_area_change['geometry_lost_early'][i]!=0:
            net_area_change['average_Farinotti_thickness_lost_early'][i]=rasterstats.zonal_stats(net_area_change['geometry_lost_early'][i],'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/RGI60-07/aa_RGI6_thickness_mosaic.vrt',stats='mean')[0]['mean']
        if net_area_change['geometry_gained_early'][i]!=0:
            net_area_change['average_Farinotti_thickness_gained_early'][i]=rasterstats.zonal_stats(net_area_change['geometry_gained_early'][i],'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/RGI60-07/aa_RGI6_thickness_mosaic.vrt',stats='mean')[0]['mean']
        if net_area_change['geometry_lost_late'][i]!=0:
            net_area_change['average_Farinotti_thickness_lost_late'][i]=rasterstats.zonal_stats(net_area_change['geometry_lost_late'][i],'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/RGI60-07/aa_RGI6_thickness_mosaic.vrt',stats='mean')[0]['mean']
        if net_area_change['geometry_gained_late'][i]!=0:
            net_area_change['average_Farinotti_thickness_gained_late'][i]=rasterstats.zonal_stats(net_area_change['geometry_gained_late'][i],'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/RGI60-07/aa_RGI6_thickness_mosaic.vrt',stats='mean')[0]['mean']
    #Russia
    if net_area_change['RGIId'][i][7]=='9':
        if net_area_change['geometry_lost_early'][i]!=0:
            net_area_change['average_Farinotti_thickness_lost_early'][i]=rasterstats.zonal_stats(net_area_change['geometry_lost_early'][i],'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/RGI60-09/aa_RGI6_thickness_mosaic.vrt',stats='mean')[0]['mean']
        if net_area_change['geometry_gained_early'][i]!=0:
            net_area_change['average_Farinotti_thickness_gained_early'][i]=rasterstats.zonal_stats(net_area_change['geometry_gained_early'][i],'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/RGI60-09/aa_RGI6_thickness_mosaic.vrt',stats='mean')[0]['mean']
        if net_area_change['geometry_lost_late'][i]!=0:
            net_area_change['average_Farinotti_thickness_lost_late'][i]=rasterstats.zonal_stats(net_area_change['geometry_lost_late'][i],'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/RGI60-09/aa_RGI6_thickness_mosaic.vrt',stats='mean')[0]['mean']
        if net_area_change['geometry_gained_late'][i]!=0:
            net_area_change['average_Farinotti_thickness_gained_late'][i]=rasterstats.zonal_stats(net_area_change['geometry_gained_late'][i],'/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Thickness_data/RGI60-09/aa_RGI6_thickness_mosaic.vrt',stats='mean')[0]['mean']

net_area_change.to_pickle('net_area_change'+date_string+'.pkl')#change file name based on distance




/Users/willkochtitzky/opt/miniconda3/envs/old_descartes/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/willkochtitzky/opt/miniconda3/envs/old_descartes/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/willkochtitzky/opt/miniconda3/envs/old_descartes/lib/python3.7/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a

/Users/willkochtitzky/opt/miniconda3/envs/old_descartes/lib/python3.7/site-packages/ipykernel_launcher.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/willkochtitzky/opt/miniconda3/envs/old_descartes/lib/python3.7/site-packages/ipykernel_launcher.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/willkochtitzky/opt/miniconda3/envs/old_descartes/lib/python3.7/site-packages/ipykernel_launcher.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a

In [46]:
#Areas that have more than 0.5 km2 area loss but dont have any thickness data for 2000-2010
net_area_change[
    (net_area_change['geometry_lost_late']!=0)&
    (net_area_change['area_km2_lost_late']>.50)&
    (net_area_change['average_Farinotti_thickness_lost_late'].isna())&
    (net_area_change['average_RUS_andrey_thickness_lost_late'].isna())&
    (net_area_change['average_RUS_julian_thickness_lost_late'].isna())&
    (net_area_change['average_tomo_thickness_lost_late'].isna())
]

,level_0,index,RGIId,Name,geometry_gained_total,area_km2_gained_total,perimeter_gained_total,perimeter_error_gained_total,geometry_lost_total,area_km2_lost_total,perimeter_lost_total,perimeter_error_lost_total,geometry_lost_early,area_km2_lost_early,perimeter_lost_early,perimeter_error_lost_early,geometry_gained_early,area_km2_gained_early,perimeter_gained_early,perimeter_error_gained_early,geometry_lost_late,area_km2_lost_late,perimeter_lost_late,perimeter_error_lost_late,geometry_gained_late,area_km2_gained_late,perimeter_gained_late,perimeter_error_gained_late,Early_satellite_id,Middle_satellite_id,Late_satellite_id,Early_date,Mid_date,Late_date,Early_date_decimal_year,Middle_date_decimal_year,Late_date_decimal_year,total_years_elapsed,early_to_middle_years_elapsed,middle_to_late_years_elapsed,net_area_change_total,net_area_change_total_error,net_area_change_per_year_total,net_area_change_per_year_total_error,net_area_change_early,net_area_change_early_error,net_area_change_per_year_early,net_area_change_per_year_early_error,net_area_change_late,net_area_change_late_error,net_area_change_per_year_late,net_area_change_per_year_late_error,average_GlaThiDa_thickness_lost_early,average_GlaThiDa_thickness_gained_early,average_GlaThiDa_thickness_lost_late,average_GlaThiDa_thickness_gained_late,average_RUS_andrey_thickness_lost_early,average_RUS_andrey_thickness_gained_early,average_RUS_andrey_thickness_lost_late,average_RUS_andrey_thickness_gained_late,average_RUS_julian_thickness_lost_early,average_RUS_julian_thickness_gained_early,average_RUS_julian_thickness_lost_late,average_RUS_julian_thickness_gained_late,average_Farinotti_thickness_lost_early,average_Farinotti_thickness_gained_early,average_Farinotti_thickness_lost_late,average_Farinotti_thickness_gained_late,average_tomo_thickness_lost_early,average_tomo_thickness_gained_early,average_tomo_thickness_lost_late,average_tomo_thickness_gained_late
261,295,295,RGI60-03.03490,0,"POLYGON ((0.00000 0.00000, 0.00000 0.00000, 0....",0.0,0.0,0.0,"POLYGON ((-794919.024 -122161.150, -794828.462...",3.343619,9612.547225,0.288376,"MULTIPOLYGON (((-792972.918 -124616.751, -7931...",0.296148,3904.072167,0.117122,"MULTIPOLYGON (((-793008.394 -124354.125, -7930...",0.818808,7676.503936,0.230295,"POLYGON ((-792752.753 -124572.718, -792972.918...",3.870774,9302.933916,0.279088,"POLYGON ((-794941.125 -122167.222, -794934.790...",0.004495,304.438027,0.009133,LE07_L1GT_057246_20000720_20170210_01_T2,AST14OTH_00306142009221941_20200729121848_1997...,LC08_L1GT_040248_20180909_20180913_01_T2,2000/07/20,2009/06/14,2018/09/09,2000.553425,2009.452055,2018.690411,18.136986,8.898630,9.238356,-3.343619,0.0,-0.184354,0.0,0.522660,0.347417,0.058735,0.039042,-3.866279,0.288221,-0.418503,0.031198,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
693,804,804,RGI60-05.10945,0,"POLYGON ((0.00000 0.00000, 0.00000 0.00000, 0....",0.0,0.0,0.0,"POLYGON ((-423519.988 -555431.133, -423484.191...",0.342940,7958.861379,0.238766,"MULTIPOLYGON (((-423519.988 -555431.133, -4235...",0.007155,1247.391300,0.037422,"MULTIPOLYGON (((-421126.402 -555074.408, -4211...",0.179708,7920.570027,0.237617,"POLYGON ((-423552.198 -555396.069, -423518.050...",0.515493,8662.198160,0.259866,"POLYGON ((0.000 0.000, 0.000 0.000, 0.000 0.00...",0.000000,0.000000,0.000000,AST_L1B_00305172003165905_20200723124405_13765...,AST_L1B_00307062010175406_20200804141857_21303...,LC08_L1GT_040245_20180808_20180815_01_T2,2003/05/17,2010/07/06,2018/08/08,2003.375342,2010.512329,2018.602740,15.227397,7.136986,8.090411,-0.342940,0.0,-0.022521,0.0,0.172554,0.275039,0.024177,0.038537,-0.515493,0.259866,-0.063717,0.032120,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
#Areas that have more than 0.5 km2 area loss but dont have any thickness data for 2010-2020
net_area_change[
    (net_area_change['geometry_lost_early']!=0)&
    (net_area_change['area_km2_lost_early']>.50)&
    (net_area_change['average_Farinotti_thickness_lost_early'].isna())&
    (net_area_change['average_RUS_andrey_thickness_lost_early'].isna())&
    (net_area_change['average_RUS_julian_thickness_lost_early'].isna())&
    (net_area_change['average_tomo_thickness_lost_early'].isna())
]

,level_0,index,RGIId,Name,geometry_gained_total,area_km2_gained_total,perimeter_gained_total,perimeter_error_gained_total,geometry_lost_total,area_km2_lost_total,perimeter_lost_total,perimeter_error_lost_total,geometry_lost_early,area_km2_lost_early,perimeter_lost_early,perimeter_error_lost_early,geometry_gained_early,area_km2_gained_early,perimeter_gained_early,perimeter_error_gained_early,geometry_lost_late,area_km2_lost_late,perimeter_lost_late,perimeter_error_lost_late,geometry_gained_late,area_km2_gained_late,perimeter_gained_late,perimeter_error_gained_late,Early_satellite_id,Middle_satellite_id,Late_satellite_id,Early_date,Mid_date,Late_date,Early_date_decimal_year,Middle_date_decimal_year,Late_date_decimal_year,total_years_elapsed,early_to_middle_years_elapsed,middle_to_late_years_elapsed,net_area_change_total,net_area_change_total_error,net_area_change_per_year_total,net_area_change_per_year_total_error,net_area_change_early,net_area_change_early_error,net_area_change_per_year_early,net_area_change_per_year_early_error,net_area_change_late,net_area_change_late_error,net_area_change_per_year_late,net_area_change_per_year_late_error,average_GlaThiDa_thickness_lost_early,average_GlaThiDa_thickness_gained_early,average_GlaThiDa_thickness_lost_late,average_GlaThiDa_thickness_gained_late,average_RUS_andrey_thickness_lost_early,average_RUS_andrey_thickness_gained_early,average_RUS_andrey_thickness_lost_late,average_RUS_andrey_thickness_gained_late,average_RUS_julian_thickness_lost_early,average_RUS_julian_thickness_gained_early,average_RUS_julian_thickness_lost_late,average_RUS_julian_thickness_gained_late,average_Farinotti_thickness_lost_early,average_Farinotti_thickness_gained_early,average_Farinotti_thickness_lost_late,average_Farinotti_thickness_gained_late,average_tomo_thickness_lost_early,average_tomo_thickness_gained_early,average_tomo_thickness_lost_late,average_tomo_thickness_gained_late
37,42,42,RGI60-01.20968,McBride Glacier,"POLYGON ((0.000 0.000, 0.000 0.000, 0.000 0.00...",0.000000,0.000000,0.000000,"POLYGON ((-2389241.759 2482529.294, -2389170.7...",3.122886,9640.609221,0.289218,"POLYGON ((-2389241.759 2482529.294, -2389170.7...",0.798842,5321.419353,0.159643,"POLYGON ((0.000 0.000, 0.000 0.000, 0.000 0.00...",0.000000,0.000000,0.000000,"POLYGON ((-2389805.061 2481196.060, -2389779.5...",2.324044,7050.915547,0.211527,"POLYGON ((0.000 0.000, 0.000 0.000, 0.000 0.00...",0.000000,0.000000,0.000000,LE07_L1TP_059019_19990801_20161003_01_T1,LT05_L1TP_060019_20090928_20160905_01_T1,LC08_L1TP_059019_20180930_20181010_01_T1,1999/08/01,2009/09/28,2018/09/30,1999.583562,2009.742466,2018.747945,19.164384,10.158904,9.005479,-3.122886,0.000000,-0.162953,0.000000,-0.798842,0.159643,-0.078635,0.015715,-2.324044,0.211527,-0.258070,0.023489,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,289.860442,NaN,NaN,NaN,NaN,NaN
257,288,288,RGI60-03.03385,0,"POLYGON ((0.000 0.000, 0.000 0.000, 0.000 0.00...",0.000000,0.000000,0.000000,"POLYGON ((-859114.722 -145623.220, -858923.775...",3.582764,8916.445285,0.267493,"POLYGON ((-859114.722 -145623.220, -858923.775...",3.506576,9462.430606,0.283873,"POLYGON ((0.000 0.000, 0.000 0.000, 0.000 0.00...",0.000000,0.000000,0.000000,"POLYGON ((-859125.849 -145644.687, -859110.206...",0.076188,3204.410168,0.096132,"POLYGON ((0.000 0.000, 0.000 0.000, 0.000 0.00...",0.000000,0.000000,0.000000,LE07_L1GT_057246_20000720_20170210_01_T2,LE07_L1TP_052247_20100814_20161213_01_T1,LC08_L1TP_068245_20190815_20190821_01_T1,2000/07/20,2010/08/14,2019/08/15,2000.553425,2010.619178,2019.621918,19.068493,10.065753,9.002740,-3.582764,0.000000,-0.187889,0.000000,-3.506576,0.283873,-0.348367,0.028202,-0.076188,0.096132,-0.008463,0.010678,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
265,302,302,RGI60-03.03547,0,"POLYGON ((0.000 0.000, 0.000 0.000, 0.000 0.00...",0.000000,0.000000,0.000000,"POLYGON ((-775405.544 -132410.597, -775164.694...",3.623110,11573.148585,0.347194,"MULT

In [20]:
#calculate the melt below the flux gate
flux_lines = gpd.read_file('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/vector_files/Arctic_marine_terminating_fluxgates.gpkg')  
#Read in RGI and fluxgate points (on and off glacier) data
RGI = gpd.read_file('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Terminus_positions/RGI_Arctic_marine_terminating_glacier_outlines.gpkg')   
RGI['geometry'] = RGI.geometry.buffer(0)
RGI['Name']=RGI['Name2']
boxes = gpd.read_file('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Terminus_positions/Glacier_area_change_boxes.gpkg')

#produce stranded portion of polygons for each glacier
for g in range(0,len(RGI)):
    for i in range(0,len(flux_lines)):
        if flux_lines.geometry.iloc[i].intersects(RGI.geometry.iloc[g]):
            split_output = split(RGI.geometry.iloc[g],flux_lines.geometry.iloc[i])#.to_file('test_split_RGI_terminus_regions',driver='GPKG')
            gdf = gpd.GeoDataFrame()
            for k in range(0,len(split_output)):
                gdf.loc[k, 'geometry'] = split_output.geoms[k]
                gdf.crs = from_epsg(3995)
                gdf.loc[k, 'area'] = split_output[k].area/1000000
            gdf = gpd.sjoin(gdf,boxes,how='inner')
                
            if g==0:
                stranded_polys = gdf.sort_values('area')[0:1]
            if g>0:
                stranded_polys = stranded_polys.append(gdf.sort_values('area')[0:1])
            
            break
stranded_polys.to_file('stranded_polygons4_'+date_string,driver='GPKG')

#attribute boxes
boxes = gpd.read_file('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Terminus_positions/Glacier_area_change_boxes.gpkg')
boxes['geometry'] = boxes['geometry'].buffer(-3)

termini_2000 = gpd.read_file('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Terminus_positions/termini_2000.geojson')
termini_2000 = gpd.overlay(termini_2000,boxes,how='intersection')
termini_2010 = gpd.read_file('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Terminus_positions/termini_2010.geojson')
termini_2010 = gpd.overlay(termini_2010,boxes,how='intersection')
termini_2020 = gpd.read_file('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/Terminus_positions/termini_2020.geojson')
termini_2020 = gpd.overlay(termini_2020,boxes,how='intersection')

boxes_attributed = gpd.sjoin(boxes, RGI, how='inner')
RGI_buffer = RGI
RGI_buffer['geometry'] = RGI_buffer['geometry'].buffer(-0.1)
max_glacier_attributed = gpd.overlay(RGI_buffer,boxes_attributed,how='union')
max_glacier_attributed['RGIid_2_1'].fillna(max_glacier_attributed['RGIid_2_2'],inplace=True)
max_glacier_attributed = max_glacier_attributed.dissolve(by='RGIid_2_1').reset_index()
max_glacier_attributed = max_glacier_attributed[['geometry','RGIid_2_1']]
max_glacier_attributed.to_file('max_glacier_attributed_v1_'+date_string,driver='GPKG')

#Get polygons that are just the area we need for computing the area below flux gate for each year
no_ice_2000 = gpd.overlay(boxes,termini_2000,how='difference')
no_ice_2000['geometry'] = no_ice_2000['geometry'].buffer(-0.1)
termini_2000_extra = gpd.overlay(termini_2000,RGI,how='difference')
standed_poly_early = gpd.overlay(termini_2000_extra,stranded_polys,how='union')
standed_poly_early = gpd.overlay(standed_poly_early,no_ice_2000,how='difference')
standed_poly_early = gpd.sjoin(standed_poly_early[['geometry']], max_glacier_attributed, how='inner')
standed_poly_early = standed_poly_early.dissolve(by='RGIid_2_1').reset_index()
standed_poly_early['geometry'] = standed_poly_early['geometry'].buffer(0.1)
standed_poly_early.to_file('stranded_polygons_early_v2_'+date_string,driver='GPKG')

no_ice_2010 = gpd.overlay(boxes,termini_2010,how='difference')
no_ice_2010['geometry'] = no_ice_2010['geometry'].buffer(-0.1)
termini_2010_extra = gpd.overlay(termini_2010,RGI,how='difference')
standed_poly_middle = gpd.overlay(termini_2010_extra,standed_poly_early,how='union')
standed_poly_middle.to_file('stranded_polygons_middle1_v2_'+date_string,driver='GPKG')
marine_based_glacier_area_lost_early = gpd.overlay(termini_2000,termini_2010,how='difference')
marine_based_glacier_area_gained_early = gpd.overlay(termini_2010,termini_2000,how='difference')
marine_based_glacier_area_lost_early.geometry = [loads(dumps(geom, rounding_precision=3)) for geom in marine_based_glacier_area_lost_early.geometry]
standed_poly_middle.geometry = [loads(dumps(geom, rounding_precision=3)) for geom in standed_poly_middle.geometry]
standed_poly_middle = gpd.overlay(standed_poly_middle,marine_based_glacier_area_lost_early,how='difference')
standed_poly_middle.to_file('stranded_polygons_middle2_v2_'+date_string,driver='GPKG')
standed_poly_middle = standed_poly_middle[(standed_poly_middle['geometry'].type=='Polygon')|(standed_poly_middle['geometry'].type=='MultiPolygon')]
standed_poly_middle.to_file('stranded_polygons_middle3_v2_'+date_string,driver='GPKG')
standed_poly_middle = gpd.overlay(marine_based_glacier_area_gained_early,standed_poly_middle,how='union')
standed_poly_middle.to_file('stranded_polygons_middle4_v2_'+date_string,driver='GPKG')
standed_poly_middle = gpd.overlay(standed_poly_middle,no_ice_2010,how='difference')
standed_poly_middle.to_file('stranded_polygons_middle5_v2_'+date_string,driver='GPKG')
standed_poly_middle['geometry'] = standed_poly_middle['geometry'].buffer(-0.1)
standed_poly_middle = gpd.sjoin(standed_poly_middle[['geometry']], max_glacier_attributed, how='inner')
standed_poly_middle = standed_poly_middle.dissolve(by='RGIid_2_1').reset_index()
standed_poly_middle['geometry'] = standed_poly_middle['geometry'].buffer(0.3)
standed_poly_middle.to_file('stranded_polygons_middle_v2_'+date_string,driver='GPKG')

no_ice_2020 = gpd.overlay(boxes,termini_2020,how='difference')
no_ice_2020['geometry'] = no_ice_2020['geometry'].buffer(-0.1)
termini_2020_extra = gpd.overlay(termini_2020,RGI,how='difference')
standed_poly_late = gpd.overlay(termini_2020_extra,standed_poly_middle,how='union')
standed_poly_late.to_file('stranded_polygons_late1_v2_'+date_string,driver='GPKG')
marine_based_glacier_area_lost_late = gpd.overlay(termini_2010,termini_2020,how='difference')
marine_based_glacier_area_gained_late = gpd.overlay(termini_2020,termini_2010,how='difference')
marine_based_glacier_area_lost_late.geometry = [loads(dumps(geom, rounding_precision=3)) for geom in marine_based_glacier_area_lost_late.geometry]
standed_poly_late.geometry = [loads(dumps(geom, rounding_precision=3)) for geom in standed_poly_late.geometry]
standed_poly_late = gpd.overlay(standed_poly_late,marine_based_glacier_area_lost_late,how='difference')
standed_poly_late.to_file('stranded_polygons_late2_v2_'+date_string,driver='GPKG')
standed_poly_late = standed_poly_late[(standed_poly_late['geometry'].type=='Polygon')|(standed_poly_late['geometry'].type=='MultiPolygon')]
standed_poly_late = gpd.overlay(marine_based_glacier_area_gained_late,standed_poly_late,how='union')
standed_poly_late.to_file('stranded_polygons_late3_v2_'+date_string,driver='GPKG')
standed_poly_late = gpd.overlay(standed_poly_late,no_ice_2020,how='difference')
standed_poly_late.to_file('stranded_polygons_late4_v2_'+date_string,driver='GPKG')
standed_poly_late['geometry'] = standed_poly_late['geometry'].buffer(-0.1)
standed_poly_late = gpd.sjoin(standed_poly_late[['geometry']], max_glacier_attributed, how='inner')
standed_poly_late.to_file('stranded_polygons_late5_v2_'+date_string,driver='GPKG')
standed_poly_late = standed_poly_late.dissolve(by='RGIid_2_1').reset_index()
standed_poly_late['geometry'] = standed_poly_late['geometry'].buffer(0.3)
standed_poly_late.to_file('stranded_polygons_late_v2_'+date_string,driver='GPKG')

/Users/willkochtitzky/opt/miniconda3/envs/old_descartes/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:3995 +no_defs +type=crs
Right CRS: EPSG:3995

/Users/willkochtitzky/opt/miniconda3/envs/old_descartes/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:3995 +no_defs +type=crs
Right CRS: EPSG:3995

/Users/willkochtitzky/opt/miniconda3/envs/old_descartes/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of th

/Users/willkochtitzky/opt/miniconda3/envs/old_descartes/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:3995 +no_defs +type=crs
Right CRS: EPSG:3995

/Users/willkochtitzky/opt/miniconda3/envs/old_descartes/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:3995 +no_defs +type=crs
Right CRS: EPSG:3995

/Users/willkochtitzky/opt/miniconda3/envs/old_descartes/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of th

/Users/willkochtitzky/opt/miniconda3/envs/old_descartes/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:3995 +no_defs +type=crs
Right CRS: EPSG:3995

/Users/willkochtitzky/opt/miniconda3/envs/old_descartes/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:3995 +no_defs +type=crs
Right CRS: EPSG:3995

/Users/willkochtitzky/opt/miniconda3/envs/old_descartes/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of th

/Users/willkochtitzky/opt/miniconda3/envs/old_descartes/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:3995 +no_defs +type=crs
Right CRS: EPSG:3995

/Users/willkochtitzky/opt/miniconda3/envs/old_descartes/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:3995 +no_defs +type=crs
Right CRS: EPSG:3995

/Users/willkochtitzky/opt/miniconda3/envs/old_descartes/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of th

/Users/willkochtitzky/opt/miniconda3/envs/old_descartes/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:3995 +no_defs +type=crs
Right CRS: EPSG:3995

/Users/willkochtitzky/opt/miniconda3/envs/old_descartes/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:3995 +no_defs +type=crs
Right CRS: EPSG:3995

/Users/willkochtitzky/opt/miniconda3/envs/old_descartes/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of th

...

/Users/willkochtitzky/opt/miniconda3/envs/old_descartes/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:3995 +no_defs +type=crs
Right CRS: EPSG:3995

/Users/willkochtitzky/opt/miniconda3/envs/old_descartes/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:3995 +no_defs +type=crs
Right CRS: EPSG:3995

/Users/willkochtitzky/opt/miniconda3/envs/old_descartes/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of th

/Users/willkochtitzky/opt/miniconda3/envs/old_descartes/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:3995 +no_defs +type=crs
Right CRS: EPSG:3995

/Users/willkochtitzky/opt/miniconda3/envs/old_descartes/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:3995 +no_defs +type=crs
Right CRS: EPSG:3995

/Users/willkochtitzky/opt/miniconda3/envs/old_descartes/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of th

/Users/willkochtitzky/opt/miniconda3/envs/old_descartes/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:3995 +no_defs +type=crs
Right CRS: EPSG:3995

/Users/willkochtitzky/opt/miniconda3/envs/old_descartes/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:3995 +no_defs +type=crs
Right CRS: EPSG:3995

/Users/willkochtitzky/opt/miniconda3/envs/old_descartes/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of th

/Users/willkochtitzky/opt/miniconda3/envs/old_descartes/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:3995 +no_defs +type=crs
Right CRS: EPSG:3995

/Users/willkochtitzky/opt/miniconda3/envs/old_descartes/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:3995 +no_defs +type=crs
Right CRS: EPSG:3995

/Users/willkochtitzky/opt/miniconda3/envs/old_descartes/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of th

/Users/willkochtitzky/opt/miniconda3/envs/old_descartes/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:3995 +no_defs +type=crs
Right CRS: EPSG:3995

/Users/willkochtitzky/opt/miniconda3/envs/old_descartes/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: +init=epsg:3995 +no_defs +type=crs
Right CRS: EPSG:3995

/Users/willkochtitzky/opt/miniconda3/envs/old_descartes/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of th

In [26]:
standed_poly_early = gpd.read_file('stranded_polygons_early_v2_'+date_string,driver='GPKG')
standed_poly_middle = gpd.read_file('stranded_polygons_middle_v2_'+date_string,driver='GPKG')
standed_poly_late = gpd.read_file('stranded_polygons_late_v2_'+date_string,driver='GPKG')

standed_poly_early['area_below_fluxgate_early']=standed_poly_early.area/1000000
standed_poly_middle['area_below_fluxgate_middle']=standed_poly_middle.area/1000000
standed_poly_late['area_below_fluxgate_late']=standed_poly_late.area/1000000

area_below_fluxgate = pd.merge(standed_poly_early,standed_poly_middle,left_on='RGIid_2_1',right_on='RGIid_2_1',how='outer',)[['RGIid_2_1','area_below_fluxgate_early','area_below_fluxgate_middle']]
area_below_fluxgate = pd.merge(area_below_fluxgate,standed_poly_late,left_on='RGIid_2_1',right_on='RGIid_2_1',how='outer',)[['RGIid_2_1','area_below_fluxgate_early','area_below_fluxgate_middle','area_below_fluxgate_late']]

Arctic_discharge_fluxes = pd.merge(Arctic_discharge_fluxes,area_below_fluxgate,left_on='RGIId',right_on='RGIid_2_1')

Arctic_discharge_fluxes_export = gpd.GeoDataFrame(Arctic_discharge_fluxes, geometry=gpd.points_from_xy(Arctic_discharge_fluxes.easting, Arctic_discharge_fluxes.northing),crs='EPSG:3995')
Arctic_discharge_fluxes_export.to_file('Arctic_discharge_fluxes_with_area_below_fluxgate.gpkg', driver="GPKG")

In [48]:
#Figure out thickness of area gain/lost
#load datasets
net_area_change = pd.read_pickle('net_area_change'+date_string+'.pkl')#read area change data so that you dont need to rerun code above
Arctic_discharge_fluxes_with_area = pd.merge(Arctic_discharge_fluxes,net_area_change,on='RGIId',how='left')#combine datasets

#Find the best thickness data for areas gained/lost by preference:
Arctic_discharge_fluxes_with_area['thickness_for_area_lost_early'] = Arctic_discharge_fluxes_with_area['average_tomo_thickness_lost_early']
Arctic_discharge_fluxes_with_area['thickness_for_area_gained_early'] = Arctic_discharge_fluxes_with_area['average_tomo_thickness_gained_early']
Arctic_discharge_fluxes_with_area['thickness_for_area_lost_late'] = Arctic_discharge_fluxes_with_area['average_tomo_thickness_lost_late']
Arctic_discharge_fluxes_with_area['thickness_for_area_gained_late'] = Arctic_discharge_fluxes_with_area['average_tomo_thickness_gained_late']

Arctic_discharge_fluxes_with_area['thickness_for_area_lost_early'].fillna(Arctic_discharge_fluxes_with_area['average_GlaThiDa_thickness_lost_early'], inplace=True)
Arctic_discharge_fluxes_with_area['thickness_for_area_gained_early'].fillna(Arctic_discharge_fluxes_with_area['average_GlaThiDa_thickness_gained_early'], inplace=True)
Arctic_discharge_fluxes_with_area['thickness_for_area_lost_late'].fillna(Arctic_discharge_fluxes_with_area['average_GlaThiDa_thickness_lost_late'], inplace=True)
Arctic_discharge_fluxes_with_area['thickness_for_area_gained_late'].fillna(Arctic_discharge_fluxes_with_area['average_GlaThiDa_thickness_gained_late'], inplace=True)

Arctic_discharge_fluxes_with_area['thickness_for_area_lost_early'].fillna(Arctic_discharge_fluxes_with_area['average_RUS_andrey_thickness_lost_early'], inplace=True)
Arctic_discharge_fluxes_with_area['thickness_for_area_gained_early'].fillna(Arctic_discharge_fluxes_with_area['average_RUS_andrey_thickness_gained_early'], inplace=True)
Arctic_discharge_fluxes_with_area['thickness_for_area_lost_late'].fillna(Arctic_discharge_fluxes_with_area['average_RUS_andrey_thickness_lost_late'], inplace=True)
Arctic_discharge_fluxes_with_area['thickness_for_area_gained_late'].fillna(Arctic_discharge_fluxes_with_area['average_RUS_andrey_thickness_gained_late'], inplace=True)

Arctic_discharge_fluxes_with_area['thickness_for_area_lost_early'].fillna(Arctic_discharge_fluxes_with_area['average_RUS_julian_thickness_lost_early'], inplace=True)
Arctic_discharge_fluxes_with_area['thickness_for_area_gained_early'].fillna(Arctic_discharge_fluxes_with_area['average_RUS_julian_thickness_gained_early'], inplace=True)
Arctic_discharge_fluxes_with_area['thickness_for_area_lost_late'].fillna(Arctic_discharge_fluxes_with_area['average_RUS_julian_thickness_lost_late'], inplace=True)
Arctic_discharge_fluxes_with_area['thickness_for_area_gained_late'].fillna(Arctic_discharge_fluxes_with_area['average_RUS_julian_thickness_gained_late'], inplace=True)

Arctic_discharge_fluxes_with_area['thickness_for_area_lost_early'].fillna(Arctic_discharge_fluxes_with_area['average_Farinotti_thickness_lost_early'], inplace=True)
Arctic_discharge_fluxes_with_area['thickness_for_area_gained_early'].fillna(Arctic_discharge_fluxes_with_area['average_Farinotti_thickness_gained_early'], inplace=True)
Arctic_discharge_fluxes_with_area['thickness_for_area_lost_late'].fillna(Arctic_discharge_fluxes_with_area['average_Farinotti_thickness_lost_late'], inplace=True)
Arctic_discharge_fluxes_with_area['thickness_for_area_gained_late'].fillna(Arctic_discharge_fluxes_with_area['average_Farinotti_thickness_gained_late'], inplace=True)

Arctic_discharge_fluxes_with_area['thickness_for_area_lost_early'].fillna(Arctic_discharge_fluxes_with_area['best_thickness_mean'], inplace=True)
Arctic_discharge_fluxes_with_area['thickness_for_area_gained_early'].fillna(Arctic_discharge_fluxes_with_area['best_thickness_mean'], inplace=True)
Arctic_discharge_fluxes_with_area['thickness_for_area_lost_late'].fillna(Arctic_discharge_fluxes_with_area['best_thickness_mean'], inplace=True)
Arctic_discharge_fluxes_with_area['thickness_for_area_gained_late'].fillna(Arctic_discharge_fluxes_with_area['best_thickness_mean'], inplace=True)

#fill in errors for thickness uncertainty
Arctic_discharge_fluxes_with_area['thickness_for_area_lost_early_err'] = Arctic_discharge_fluxes_with_area['thickness_for_area_lost_early']*.25
Arctic_discharge_fluxes_with_area['thickness_for_area_gained_early_err'] = Arctic_discharge_fluxes_with_area['thickness_for_area_gained_early']*.25
Arctic_discharge_fluxes_with_area['thickness_for_area_lost_late_err'] = Arctic_discharge_fluxes_with_area['thickness_for_area_lost_late']*.25
Arctic_discharge_fluxes_with_area['thickness_for_area_gained_late_err'] = Arctic_discharge_fluxes_with_area['thickness_for_area_gained_late']*.25


#Use the fluxgate mean thickness instead of Farinotti results if we have observations at the fluxgate
for i in range(0,len(Arctic_discharge_fluxes_with_area)):
#     print(i)
#     print(Arctic_discharge_fluxes_with_area['RGIId'][i])
    if (Arctic_discharge_fluxes_with_area['do_we_have_an_observation_in_middle_fifth'][i] ==1)&\
    (math.isnan(Arctic_discharge_fluxes_with_area['average_tomo_thickness_lost_early'][i]))&\
    (math.isnan(Arctic_discharge_fluxes_with_area['average_GlaThiDa_thickness_lost_early'][i]))&\
    (math.isnan(Arctic_discharge_fluxes_with_area['average_RUS_andrey_thickness_lost_early'][i]))&\
    (math.isnan(Arctic_discharge_fluxes_with_area['average_RUS_julian_thickness_lost_early'][i])):
        Arctic_discharge_fluxes_with_area['thickness_for_area_lost_early'][i]=Arctic_discharge_fluxes_with_area['best_thickness_mean'][i]
    if (Arctic_discharge_fluxes_with_area['do_we_have_an_observation_in_middle_fifth'][i] ==1)&\
    (math.isnan(Arctic_discharge_fluxes_with_area['average_tomo_thickness_gained_early'][i]))&\
    (math.isnan(Arctic_discharge_fluxes_with_area['average_GlaThiDa_thickness_gained_early'][i]))&\
    (math.isnan(Arctic_discharge_fluxes_with_area['average_RUS_andrey_thickness_gained_early'][i]))&\
    (math.isnan(Arctic_discharge_fluxes_with_area['average_RUS_julian_thickness_gained_early'][i])):
        Arctic_discharge_fluxes_with_area['thickness_for_area_gained_early'][i]=Arctic_discharge_fluxes_with_area['best_thickness_mean'][i]
    if (Arctic_discharge_fluxes_with_area['do_we_have_an_observation_in_middle_fifth'][i] ==1)&\
    (math.isnan(Arctic_discharge_fluxes_with_area['average_tomo_thickness_lost_late'][i]))&\
    (math.isnan(Arctic_discharge_fluxes_with_area['average_GlaThiDa_thickness_lost_late'][i]))&\
    (math.isnan(Arctic_discharge_fluxes_with_area['average_RUS_andrey_thickness_lost_late'][i]))&\
    (math.isnan(Arctic_discharge_fluxes_with_area['average_RUS_julian_thickness_lost_late'][i])):
        Arctic_discharge_fluxes_with_area['thickness_for_area_lost_late'][i]=Arctic_discharge_fluxes_with_area['best_thickness_mean'][i]
    if (Arctic_discharge_fluxes_with_area['do_we_have_an_observation_in_middle_fifth'][i] ==1)&\
    (math.isnan(Arctic_discharge_fluxes_with_area['average_tomo_thickness_gained_late'][i]))&\
    (math.isnan(Arctic_discharge_fluxes_with_area['average_GlaThiDa_thickness_gained_late'][i]))&\
    (math.isnan(Arctic_discharge_fluxes_with_area['average_RUS_andrey_thickness_gained_late'][i]))&\
    (math.isnan(Arctic_discharge_fluxes_with_area['average_RUS_julian_thickness_gained_late'][i])):
        Arctic_discharge_fluxes_with_area['thickness_for_area_gained_late'][i]=Arctic_discharge_fluxes_with_area['best_thickness_mean'][i]

#Merge area change data for every glacier with the discharge estimates
#calculate discharge due to terminus retreat/advance with uncertainties (change thickness uncertainty above)
Arctic_discharge_fluxes_with_area['area_flux_gt_gained_early'] = Arctic_discharge_fluxes_with_area['area_km2_gained_early']*Arctic_discharge_fluxes_with_area['thickness_for_area_gained_early']/1000*0.900
Arctic_discharge_fluxes_with_area['area_flux_gt_gained_early_error'] = \
    (((Arctic_discharge_fluxes_with_area['area_km2_gained_early'] + Arctic_discharge_fluxes_with_area['perimeter_error_gained_early'])*\
    (Arctic_discharge_fluxes_with_area['thickness_for_area_gained_early']/1000 + Arctic_discharge_fluxes_with_area['thickness_for_area_gained_early_err']/1000))-\
    ((Arctic_discharge_fluxes_with_area['area_km2_gained_early'] - Arctic_discharge_fluxes_with_area['perimeter_error_gained_early'])*\
    (Arctic_discharge_fluxes_with_area['thickness_for_area_gained_early']/1000 - Arctic_discharge_fluxes_with_area['thickness_for_area_gained_early_err']/1000)))/2*0.900

Arctic_discharge_fluxes_with_area['area_flux_gt_lost_early'] = Arctic_discharge_fluxes_with_area['area_km2_lost_early']*Arctic_discharge_fluxes_with_area['thickness_for_area_lost_early']/1000*0.900
Arctic_discharge_fluxes_with_area['area_flux_gt_lost_early_error'] = \
    (((Arctic_discharge_fluxes_with_area['area_km2_lost_early'] + Arctic_discharge_fluxes_with_area['perimeter_error_lost_early'])*\
    (Arctic_discharge_fluxes_with_area['thickness_for_area_lost_early']/1000 + Arctic_discharge_fluxes_with_area['thickness_for_area_lost_early_err']/1000))-\
    ((Arctic_discharge_fluxes_with_area['area_km2_lost_early'] - Arctic_discharge_fluxes_with_area['perimeter_error_lost_early'])*\
    (Arctic_discharge_fluxes_with_area['thickness_for_area_lost_early']/1000 - Arctic_discharge_fluxes_with_area['thickness_for_area_lost_early_err']/1000)))/2*0.900

Arctic_discharge_fluxes_with_area['area_flux_gt_gained_late'] = Arctic_discharge_fluxes_with_area['area_km2_gained_late']*Arctic_discharge_fluxes_with_area['thickness_for_area_gained_late']/1000*0.900
Arctic_discharge_fluxes_with_area['area_flux_gt_gained_late_error'] = \
    (((Arctic_discharge_fluxes_with_area['area_km2_gained_late'] + Arctic_discharge_fluxes_with_area['perimeter_error_gained_late'])*\
    (Arctic_discharge_fluxes_with_area['thickness_for_area_gained_late']/1000 + Arctic_discharge_fluxes_with_area['thickness_for_area_gained_late_err']/1000))-\
    ((Arctic_discharge_fluxes_with_area['area_km2_gained_late'] - Arctic_discharge_fluxes_with_area['perimeter_error_gained_late'])*\
    (Arctic_discharge_fluxes_with_area['thickness_for_area_gained_late']/1000 - Arctic_discharge_fluxes_with_area['thickness_for_area_gained_late_err']/1000)))/2*0.900

Arctic_discharge_fluxes_with_area['area_flux_gt_lost_late'] = Arctic_discharge_fluxes_with_area['area_km2_lost_late']*Arctic_discharge_fluxes_with_area['thickness_for_area_lost_late']/1000*0.900
Arctic_discharge_fluxes_with_area['area_flux_gt_lost_late_error'] = \
    (((Arctic_discharge_fluxes_with_area['area_km2_lost_late'] + Arctic_discharge_fluxes_with_area['perimeter_error_lost_late'])*\
    (Arctic_discharge_fluxes_with_area['thickness_for_area_lost_late_err']/1000 + Arctic_discharge_fluxes_with_area['thickness_for_area_lost_late_err']/1000))-\
    ((Arctic_discharge_fluxes_with_area['area_km2_lost_late'] - Arctic_discharge_fluxes_with_area['perimeter_error_lost_late'])*\
    (Arctic_discharge_fluxes_with_area['thickness_for_area_lost_late_err']/1000 - Arctic_discharge_fluxes_with_area['thickness_for_area_lost_late_err']/1000)))/2*0.900



Arctic_discharge_fluxes_with_area['terminus_gt_change_total']=Arctic_discharge_fluxes_with_area['net_area_change_total']*Arctic_discharge_fluxes_with_area['best_thickness_mean']/1000*0.900
Arctic_discharge_fluxes_with_area['terminus_gt_change_total_area_error']=Arctic_discharge_fluxes_with_area['net_area_change_total_error']*Arctic_discharge_fluxes_with_area['best_thickness_mean']/1000*0.900
Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_total']=Arctic_discharge_fluxes_with_area['net_area_change_per_year_total']*Arctic_discharge_fluxes_with_area['best_thickness_mean']/1000*0.900
Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_total_area_error']=Arctic_discharge_fluxes_with_area['net_area_change_per_year_total_error']*Arctic_discharge_fluxes_with_area['best_thickness_mean']/1000*0.900


Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_early']=(Arctic_discharge_fluxes_with_area['area_flux_gt_gained_early']/Arctic_discharge_fluxes_with_area['early_to_middle_years_elapsed'])-(Arctic_discharge_fluxes_with_area['area_flux_gt_lost_early']/Arctic_discharge_fluxes_with_area['early_to_middle_years_elapsed'])
Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_early_area_error']=np.sqrt((Arctic_discharge_fluxes_with_area['area_flux_gt_gained_early_error']/Arctic_discharge_fluxes_with_area['early_to_middle_years_elapsed'])**2 + (Arctic_discharge_fluxes_with_area['area_flux_gt_lost_early_error']/Arctic_discharge_fluxes_with_area['early_to_middle_years_elapsed'])**2)
Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_late']=(Arctic_discharge_fluxes_with_area['area_flux_gt_gained_late']/Arctic_discharge_fluxes_with_area['middle_to_late_years_elapsed'])-(Arctic_discharge_fluxes_with_area['area_flux_gt_lost_late']/Arctic_discharge_fluxes_with_area['middle_to_late_years_elapsed'])
Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_late_area_error']=np.sqrt((Arctic_discharge_fluxes_with_area['area_flux_gt_gained_late_error']/Arctic_discharge_fluxes_with_area['middle_to_late_years_elapsed'])**2 + (Arctic_discharge_fluxes_with_area['area_flux_gt_lost_late_error']/Arctic_discharge_fluxes_with_area['middle_to_late_years_elapsed'])**2)

#If the errors are bigger than the signal, signal=error
for i in range(0,len(Arctic_discharge_fluxes_with_area)):
    if Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_early_area_error'][i]>np.abs(Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_early'][i]):
        Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_early_area_error'][i]=np.abs(Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_early'][i])
    if Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_late_area_error'][i]>np.abs(Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_late'][i]):
        Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_late_area_error'][i]=np.abs(Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_late'][i])

#calculate final numbers
#sqrt on uncertainties because of this link: http://lectureonline.cl.msu.edu/~mmp/labs/error/e2.htm
Arctic_discharge_fluxes_with_area['Frontal_ablation_2000_to_2010_gt_per_yr_mean'] = -Arctic_discharge_fluxes_with_area['2000_2010_mean_flux_gt'] + Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_early']
Arctic_discharge_fluxes_with_area['Frontal_ablation_2000_to_2010_gt_per_yr_mean_err'] = np.sqrt(Arctic_discharge_fluxes_with_area['2000_2010_mean_err_gt']**2 + Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_early_area_error']**2)
Arctic_discharge_fluxes_with_area['Frontal_ablation_2010_to_2020_gt_per_yr_mean'] = -Arctic_discharge_fluxes_with_area['2010_2018_mean_flux_gt'] + Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_late']
Arctic_discharge_fluxes_with_area['Frontal_ablation_2010_to_2020_gt_per_yr_mean_err'] = np.sqrt(Arctic_discharge_fluxes_with_area['2010_2018_mean_err_gt']**2 + Arctic_discharge_fluxes_with_area['terminus_gt_change_per_year_late_area_error']**2)

#Add in Jan Mayen manualy since we have no data there
#Weyprechtbreen
mask = Arctic_discharge_fluxes_with_area['RGIId']=='RGI60-07.01575'
Arctic_discharge_fluxes_with_area.loc[mask, 'Frontal_ablation_2000_to_2010_gt_per_yr_mean']=Arctic_discharge_fluxes_with_area.loc[mask, 'Frontal_ablation_2000_to_2010_gt_per_yr_mean'].replace(0,-0.00687)
Arctic_discharge_fluxes_with_area.loc[mask, 'Frontal_ablation_2000_to_2010_gt_per_yr_mean_err']=Arctic_discharge_fluxes_with_area.loc[mask, 'Frontal_ablation_2000_to_2010_gt_per_yr_mean_err'].replace(0,0.00483)
Arctic_discharge_fluxes_with_area.loc[mask, 'Frontal_ablation_2010_to_2020_gt_per_yr_mean']=Arctic_discharge_fluxes_with_area.loc[mask, 'Frontal_ablation_2010_to_2020_gt_per_yr_mean'].replace(0,-0.00687)
Arctic_discharge_fluxes_with_area.loc[mask, 'Frontal_ablation_2010_to_2020_gt_per_yr_mean_err']=Arctic_discharge_fluxes_with_area.loc[mask, 'Frontal_ablation_2010_to_2020_gt_per_yr_mean_err'].replace(0,0.00483)
#Kjerulfbreen
mask = Arctic_discharge_fluxes_with_area['RGIId']=='RGI60-07.01577'
Arctic_discharge_fluxes_with_area.loc[mask, 'Frontal_ablation_2000_to_2010_gt_per_yr_mean']=Arctic_discharge_fluxes_with_area.loc[mask, 'Frontal_ablation_2000_to_2010_gt_per_yr_mean'].replace(0,-0.00550)
Arctic_discharge_fluxes_with_area.loc[mask, 'Frontal_ablation_2000_to_2010_gt_per_yr_mean_err']=Arctic_discharge_fluxes_with_area.loc[mask, 'Frontal_ablation_2000_to_2010_gt_per_yr_mean_err'].replace(0,0.00381)
Arctic_discharge_fluxes_with_area.loc[mask, 'Frontal_ablation_2010_to_2020_gt_per_yr_mean']=Arctic_discharge_fluxes_with_area.loc[mask, 'Frontal_ablation_2010_to_2020_gt_per_yr_mean'].replace(0,-0.00550)
Arctic_discharge_fluxes_with_area.loc[mask, 'Frontal_ablation_2010_to_2020_gt_per_yr_mean_err']=Arctic_discharge_fluxes_with_area.loc[mask, 'Frontal_ablation_2010_to_2020_gt_per_yr_mean_err'].replace(0,0.00381)
#Pr Haralds Breen
mask = Arctic_discharge_fluxes_with_area['RGIId']=='RGI60-07.01582'
Arctic_discharge_fluxes_with_area.loc[mask, 'Frontal_ablation_2000_to_2010_gt_per_yr_mean']=Arctic_discharge_fluxes_with_area.loc[mask, 'Frontal_ablation_2000_to_2010_gt_per_yr_mean'].replace(0,-0.00312)
Arctic_discharge_fluxes_with_area.loc[mask, 'Frontal_ablation_2000_to_2010_gt_per_yr_mean_err']=Arctic_discharge_fluxes_with_area.loc[mask, 'Frontal_ablation_2000_to_2010_gt_per_yr_mean_err'].replace(0,0.00252)
Arctic_discharge_fluxes_with_area.loc[mask, 'Frontal_ablation_2010_to_2020_gt_per_yr_mean']=Arctic_discharge_fluxes_with_area.loc[mask, 'Frontal_ablation_2010_to_2020_gt_per_yr_mean'].replace(0,-0.00312)
Arctic_discharge_fluxes_with_area.loc[mask, 'Frontal_ablation_2010_to_2020_gt_per_yr_mean_err']=Arctic_discharge_fluxes_with_area.loc[mask, 'Frontal_ablation_2010_to_2020_gt_per_yr_mean_err'].replace(0,0.00252)
#Willebreen
mask = Arctic_discharge_fluxes_with_area['RGIId']=='RGI60-07.01584'
Arctic_discharge_fluxes_with_area.loc[mask, 'Frontal_ablation_2000_to_2010_gt_per_yr_mean']=Arctic_discharge_fluxes_with_area.loc[mask, 'Frontal_ablation_2000_to_2010_gt_per_yr_mean'].replace(0,-0.00295)
Arctic_discharge_fluxes_with_area.loc[mask, 'Frontal_ablation_2000_to_2010_gt_per_yr_mean_err']=Arctic_discharge_fluxes_with_area.loc[mask, 'Frontal_ablation_2000_to_2010_gt_per_yr_mean_err'].replace(0,0.00238)
Arctic_discharge_fluxes_with_area.loc[mask, 'Frontal_ablation_2010_to_2020_gt_per_yr_mean']=Arctic_discharge_fluxes_with_area.loc[mask, 'Frontal_ablation_2010_to_2020_gt_per_yr_mean'].replace(0,-0.00295)
Arctic_discharge_fluxes_with_area.loc[mask, 'Frontal_ablation_2010_to_2020_gt_per_yr_mean_err']=Arctic_discharge_fluxes_with_area.loc[mask, 'Frontal_ablation_2010_to_2020_gt_per_yr_mean_err'].replace(0,0.00238)
#Frielebreen
mask = Arctic_discharge_fluxes_with_area['RGIId']=='RGI60-07.01581'
Arctic_discharge_fluxes_with_area.loc[mask, 'Frontal_ablation_2000_to_2010_gt_per_yr_mean']=Arctic_discharge_fluxes_with_area.loc[mask, 'Frontal_ablation_2000_to_2010_gt_per_yr_mean'].replace(0,-0.00158)
Arctic_discharge_fluxes_with_area.loc[mask, 'Frontal_ablation_2000_to_2010_gt_per_yr_mean_err']=Arctic_discharge_fluxes_with_area.loc[mask, 'Frontal_ablation_2000_to_2010_gt_per_yr_mean_err'].replace(0,0.00128)
Arctic_discharge_fluxes_with_area.loc[mask, 'Frontal_ablation_2010_to_2020_gt_per_yr_mean']=Arctic_discharge_fluxes_with_area.loc[mask, 'Frontal_ablation_2010_to_2020_gt_per_yr_mean'].replace(0,-0.00158)
Arctic_discharge_fluxes_with_area.loc[mask, 'Frontal_ablation_2010_to_2020_gt_per_yr_mean_err']=Arctic_discharge_fluxes_with_area.loc[mask, 'Frontal_ablation_2010_to_2020_gt_per_yr_mean_err'].replace(0,0.00128)
#Svend Foynbreen
mask = Arctic_discharge_fluxes_with_area['RGIId']=='RGI60-07.01578'
Arctic_discharge_fluxes_with_area.loc[mask, 'Frontal_ablation_2000_to_2010_gt_per_yr_mean']=Arctic_discharge_fluxes_with_area.loc[mask, 'Frontal_ablation_2000_to_2010_gt_per_yr_mean'].replace(0,-0.00044)
Arctic_discharge_fluxes_with_area.loc[mask, 'Frontal_ablation_2000_to_2010_gt_per_yr_mean_err']=Arctic_discharge_fluxes_with_area.loc[mask, 'Frontal_ablation_2000_to_2010_gt_per_yr_mean_err'].replace(0,0.00186)
Arctic_discharge_fluxes_with_area.loc[mask, 'Frontal_ablation_2010_to_2020_gt_per_yr_mean']=Arctic_discharge_fluxes_with_area.loc[mask, 'Frontal_ablation_2010_to_2020_gt_per_yr_mean'].replace(0,-0.00044)
Arctic_discharge_fluxes_with_area.loc[mask, 'Frontal_ablation_2010_to_2020_gt_per_yr_mean_err']=Arctic_discharge_fluxes_with_area.loc[mask, 'Frontal_ablation_2010_to_2020_gt_per_yr_mean_err'].replace(0,0.00186)

Arctic_discharge_fluxes_with_area=Arctic_discharge_fluxes_with_area.drop(columns=['Name_y', 'geometry_gained_total','geometry_lost_total', 'geometry_gained_early','geometry_lost_early', 'geometry_gained_late','geometry_lost_late'])
#export data with flux and area change estimates
easting=[]
northing=[]
for i in range(0,len(Arctic_discharge_fluxes)):
    easting.append(Arctic_discharge_fluxes_with_area['easting'][i])
    northing.append(Arctic_discharge_fluxes_with_area['northing'][i])

#add lat lon in decimal degrees
inProj = Proj(init='epsg:3995')
outProj = Proj(init='epsg:4326')    
Arctic_discharge_fluxes_with_area['lon'],Arctic_discharge_fluxes_with_area['lat'] = transform(inProj,outProj,easting,northing)
    
# Arctic_discharge_fluxes_with_area.to_csv('Arctic_discharge_fluxes_with_area_'+date_string+'.txt')

#export to geopackage
Arctic_discharge_fluxes_with_area_export = gpd.GeoDataFrame(Arctic_discharge_fluxes_with_area, geometry=gpd.points_from_xy(Arctic_discharge_fluxes_with_area.easting, Arctic_discharge_fluxes_with_area.northing),crs='EPSG:3995')
#Add regions as an attribute
arctic_aois = gpd.read_file('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/vector_files/Arctic_AOIs.gpkg')
arctic_subaois = gpd.read_file('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/vector_files/Arctic_subAOIs.gpkg')
arctic_subaois2 = gpd.read_file('/Users/willkochtitzky/Projects/Arctic_frontal_ablation/vector_files/Arctic_subAOIs_v2.gpkg')
Arctic_discharge_fluxes_with_area_export = gpd.sjoin(Arctic_discharge_fluxes_with_area_export,arctic_aois)
Arctic_discharge_fluxes_with_area_export = Arctic_discharge_fluxes_with_area_export.drop(columns='index_right')
Arctic_discharge_fluxes_with_area_export = gpd.sjoin(Arctic_discharge_fluxes_with_area_export,arctic_subaois)
Arctic_discharge_fluxes_with_area_export = Arctic_discharge_fluxes_with_area_export.drop(columns='index_right')
Arctic_discharge_fluxes_with_area_export = gpd.sjoin(Arctic_discharge_fluxes_with_area_export,arctic_subaois2)
Arctic_discharge_fluxes_with_area_export = Arctic_discharge_fluxes_with_area_export.drop(columns='index_right')


print('Numbers of glaciers with a positive FA during 2000-2010',len(Arctic_discharge_fluxes_with_area_export[Arctic_discharge_fluxes_with_area_export['Frontal_ablation_2000_to_2010_gt_per_yr_mean']>0]))
print(Arctic_discharge_fluxes_with_area_export[Arctic_discharge_fluxes_with_area_export['Frontal_ablation_2000_to_2010_gt_per_yr_mean']>0][['Name_x','terminus_gt_change_per_year_early']])
print('Numbers of glaciers with a positive FA during 2010-2020',len(Arctic_discharge_fluxes_with_area_export[Arctic_discharge_fluxes_with_area_export['Frontal_ablation_2010_to_2020_gt_per_yr_mean']>0]))
print(Arctic_discharge_fluxes_with_area_export[Arctic_discharge_fluxes_with_area_export['Frontal_ablation_2010_to_2020_gt_per_yr_mean']>0][['Name_x','terminus_gt_change_per_year_late']])

#if the FA is greater than 0, force it to be 0 and force terminus change to equal flux
for i in range(0,len(Arctic_discharge_fluxes_with_area_export)):
    if Arctic_discharge_fluxes_with_area_export['Frontal_ablation_2000_to_2010_gt_per_yr_mean'][i]>0:
        Arctic_discharge_fluxes_with_area_export['Frontal_ablation_2000_to_2010_gt_per_yr_mean'][i]=((Arctic_discharge_fluxes_with_area_export['2000_2010_mean_flux_gt'][i]+Arctic_discharge_fluxes_with_area_export['2000_2010_mean_err_gt'][i])-(Arctic_discharge_fluxes_with_area_export['terminus_gt_change_per_year_early'][i]-Arctic_discharge_fluxes_with_area_export['terminus_gt_change_per_year_early_area_error'][i]))/2    
        Arctic_discharge_fluxes_with_area_export['Frontal_ablation_2000_to_2010_gt_per_yr_mean_err'][i]=Arctic_discharge_fluxes_with_area_export['Frontal_ablation_2000_to_2010_gt_per_yr_mean'][i] #error will be same as signal
        Arctic_discharge_fluxes_with_area_export['terminus_gt_change_per_year_early'][i] = Arctic_discharge_fluxes_with_area_export['2000_2010_mean_flux_gt'][i]
        Arctic_discharge_fluxes_with_area_export['terminus_gt_change_per_year_early_area_error'][i] = Arctic_discharge_fluxes_with_area_export['2000_2010_mean_err_gt'][i]
        if Arctic_discharge_fluxes_with_area_export['Frontal_ablation_2000_to_2010_gt_per_yr_mean'][i]>0:
            Arctic_discharge_fluxes_with_area_export['Frontal_ablation_2000_to_2010_gt_per_yr_mean'][i]=0
            Arctic_discharge_fluxes_with_area_export['Frontal_ablation_2000_to_2010_gt_per_yr_mean_err'][i]=0
    if Arctic_discharge_fluxes_with_area_export['Frontal_ablation_2010_to_2020_gt_per_yr_mean'][i]>0:
        Arctic_discharge_fluxes_with_area_export['Frontal_ablation_2010_to_2020_gt_per_yr_mean'][i]=((Arctic_discharge_fluxes_with_area_export['2010_2018_mean_flux_gt'][i]+Arctic_discharge_fluxes_with_area_export['2010_2018_mean_err_gt'][i])-(Arctic_discharge_fluxes_with_area_export['terminus_gt_change_per_year_late'][i]-Arctic_discharge_fluxes_with_area_export['terminus_gt_change_per_year_late_area_error'][i]))/2    
        Arctic_discharge_fluxes_with_area_export['Frontal_ablation_2010_to_2020_gt_per_yr_mean_err'][i]=Arctic_discharge_fluxes_with_area_export['Frontal_ablation_2010_to_2020_gt_per_yr_mean'][i] #error will be same as signal
        Arctic_discharge_fluxes_with_area_export['terminus_gt_change_per_year_late'][i] = Arctic_discharge_fluxes_with_area_export['2010_2018_mean_flux_gt'][i]
        Arctic_discharge_fluxes_with_area_export['terminus_gt_change_per_year_late_area_error'][i] = Arctic_discharge_fluxes_with_area_export['2010_2018_mean_err_gt'][i]
        if Arctic_discharge_fluxes_with_area_export['Frontal_ablation_2010_to_2020_gt_per_yr_mean'][i]>0:
            Arctic_discharge_fluxes_with_area_export['Frontal_ablation_2010_to_2020_gt_per_yr_mean'][i]=0
            Arctic_discharge_fluxes_with_area_export['Frontal_ablation_2010_to_2020_gt_per_yr_mean_err'][i]=0
        
Arctic_discharge_fluxes_with_area_export.to_file('Arctic_discharge_fluxes_with_area_epsg3995_'+date_string+'_V4.gpkg', driver="GPKG")#change file name based on distance
# Arctic_discharge_fluxes_with_area_export

/Users/willkochtitzky/opt/miniconda3/envs/old_descartes/lib/python3.7/site-packages/ipykernel_launcher.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/willkochtitzky/opt/miniconda3/envs/old_descartes/lib/python3.7/site-packages/ipykernel_launcher.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/willkochtitzky/opt/miniconda3/envs/old_descartes/lib/python3.7/site-packages/ipykernel_launcher.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a

Numbers of glaciers with a positive FA during 2000-2010 16
                                                 Name_x  \
25                                       Yahtse Glacier   
47                                              Parrish   
49                                           Dobbin Bay   
233                                                 NaN   
247                                                 NaN   
261                                                 NaN   
268                                                 NaN   
688                                                 NaN   
693                                                 NaN   
700                                                 NaN   
704                                                 NaN   
705                                                 NaN   
706                                                 NaN   
929   Dobrowolskibreen, Nathorstbreen, Zawadskibreen...   
1023             Bakaninbreen, Paulabreen, and Skobreen 

/Users/willkochtitzky/opt/miniconda3/envs/old_descartes/lib/python3.7/site-packages/ipykernel_launcher.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/willkochtitzky/opt/miniconda3/envs/old_descartes/lib/python3.7/site-packages/ipykernel_launcher.py:206: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/willkochtitzky/opt/miniconda3/envs/old_descartes/lib/python3.7/site-packages/ipykernel_launcher.py:207: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returnin

In [49]:
for i in range(0,len(Arctic_discharge_fluxes_with_area_export)):
    if Arctic_discharge_fluxes_with_area_export['Frontal_ablation_2010_to_2020_gt_per_yr_mean'][i]==0:
        print(i)
        
        

172
173
273
279
280
690
929
1423


In [50]:
Arctic_discharge_fluxes_with_area_export[Arctic_discharge_fluxes_with_area_export['Frontal_ablation_2010_to_2020_gt_per_yr_mean']==0]

,RGIId,flux_min,flux_max,2000_2010_mean_flux_min,2000_2010_mean_flux_max,2010_2018_mean_flux_min,2010_2018_mean_flux_max,Name_x,Area,Zmin,Zmax,Zmed,Slope,Aspect,Lmax,Status,Connect,Form,TermType,Surging,Linkages,Farinotti_thickness_min,region,Farinotti_thickness_max,do_we_have_an_observation_in_middle_fifth,easting,northing,Farinotti_thickness_mean,0000_v,best_thickness_mean,thickness_mean_uncertainty,length_early_min,length_early_max,length_late_min,length_late_max,2000_2010_mean,2000_2010_mean_err,2010_2018_mean,2010_2018_mean_err,num_pts,GlaThiDa_observations,Tomography_observations,Russian_observations,gate_length_km,IL_flux_gt,IL_err_gt,2000_2010_mean_flux_gt,2000_2010_mean_err_gt,2010_2018_mean_flux_gt,2010_2018_mean_err_gt,level_0,index,area_km2_gained_total,perimeter_gained_total,perimeter_error_gained_total,area_km2_lost_total,perimeter_lost_total,perimeter_error_lost_total,area_km2_lost_early,perimeter_lost_early,perimeter_error_lost_early,area_km2_gained_early,perimeter_gained_early,perimeter_error_gained_early,area_km2_lost_late,perimeter_lost_late,perimeter_error_lost_late,area_km2_gained_late,perimeter_gained_late,perimeter_error_gained_late,Early_satellite_id,Middle_satellite_id,Late_satellite_id,Early_date,Mid_date,Late_date,Early_date_decimal_year,Middle_date_decimal_year,Late_date_decimal_year,total_years_elapsed,early_to_middle_years_elapsed,middle_to_late_years_elapsed,net_area_change_total,net_area_change_total_error,net_area_change_per_year_total,net_area_change_per_year_total_error,net_area_change_early,net_area_change_early_error,net_area_change_per_year_early,net_area_change_per_year_early_error,net_area_change_late,net_area_change_late_error,net_area_change_per_year_late,net_area_change_per_year_late_error,average_GlaThiDa_thickness_lost_early,average_GlaThiDa_thickness_gained_early,average_GlaThiDa_thickness_lost_late,average_GlaThiDa_thickness_gained_late,average_RUS_andrey_thickness_lost_early,average_RUS_andrey_thickness_gained_early,average_RUS_andrey_thickness_lost_late,average_RUS_andrey_thickness_gained_late,average_RUS_julian_thickness_lost_early,average_RUS_julian_thickness_gained_early,average_RUS_julian_thickness_lost_late,average_RUS_julian_thickness_gained_late,average_Farinotti_thickness_lost_early,average_Farinotti_thickness_gained_early,average_Farinotti_thickness_lost_late,average_Farinotti_thickness_gained_late,average_tomo_thickness_lost_early,average_tomo_thickness_gained_early,average_tomo_thickness_lost_late,average_tomo_thickness_gained_late,thickness_for_area_lost_early,thickness_for_area_gained_early,thickness_for_area_lost_late,thickness_for_area_gained_late,thickness_for_area_lost_early_err,thickness_for_area_gained_early_err,thickness_for_area_lost_late_err,thickness_for_area_gained_late_err,area_flux_gt_gained_early,area_flux_gt_gained_early_error,area_flux_gt_lost_early,area_flux_gt_lost_early_error,area_flux_gt_gained_late,area_flux_gt_gained_late_error,area_flux_gt_lost_late,area_flux_gt_lost_late_error,terminus_gt_change_total,terminus_gt_change_total_area_error,terminus_gt_change_per_year_total,terminus_gt_change_per_year_total_area_error,terminus_gt_change_per_year_early,terminus_gt_change_per_year_early_area_error,terminus_gt_change_per_year_late,terminus_gt_change_per_year_late_area_error,Frontal_ablation_2000_to_2010_gt_per_yr_mean,Frontal_ablation_2000_to_2010_gt_per_yr_mean_err,Frontal_ablation_2010_to_2020_gt_per_yr_mean,Frontal_ablation_2010_to_2020_gt_per_yr_mean_err,lon,lat,geometry,Region_name,Subregion_name,Subregion_name2
172,RGI60-03.01740,0.000007,0.000028,0.000007,0.000109,0.000012,0.000071,NaN,39.737,36.0,1345.0,1249.0,14.2,160.0,9888.0,0.0,0.0,1.0,1.0,0.0,9.0,44.636955,Canada,86.558235,NaN,-1.213080e+06,-2.943009e+05,67.427933,1.010907,67.427933,17.531263,17.000000,25.000000,17.000000,25.000000,2.731286,1.962036,2.240536,0.787163,11,0,0,0,0.275,0.000016,0.000010,0.000053,0.000047,0.000038,0.000027,193,193,0.012125,727.672676,0.021830,0.014946,1842.8

In [51]:
Arctic_discharge_fluxes_with_area_export.sum()[100:]

average_RUS_andrey_thickness_gained_late                                                      189.161
average_RUS_julian_thickness_lost_early                                                       612.303
average_RUS_julian_thickness_gained_early                                                         126
average_RUS_julian_thickness_lost_late                                                        924.691
average_RUS_julian_thickness_gained_late                                                        176.5
average_Farinotti_thickness_lost_early                                                        87343.1
average_Farinotti_thickness_gained_early                                                      18764.7
average_Farinotti_thickness_lost_late                                                          115388
average_Farinotti_thickness_gained_late                                                       17563.4
average_tomo_thickness_lost_early                                                 

In [52]:
Arctic_discharge_fluxes_with_area_export[Arctic_discharge_fluxes_with_area_export['Frontal_ablation_2000_to_2010_gt_per_yr_mean']<-1.5]

,RGIId,flux_min,flux_max,2000_2010_mean_flux_min,2000_2010_mean_flux_max,2010_2018_mean_flux_min,2010_2018_mean_flux_max,Name_x,Area,Zmin,Zmax,Zmed,Slope,Aspect,Lmax,Status,Connect,Form,TermType,Surging,Linkages,Farinotti_thickness_min,region,Farinotti_thickness_max,do_we_have_an_observation_in_middle_fifth,easting,northing,Farinotti_thickness_mean,0000_v,best_thickness_mean,thickness_mean_uncertainty,length_early_min,length_early_max,length_late_min,length_late_max,2000_2010_mean,2000_2010_mean_err,2010_2018_mean,2010_2018_mean_err,num_pts,GlaThiDa_observations,Tomography_observations,Russian_observations,gate_length_km,IL_flux_gt,IL_err_gt,2000_2010_mean_flux_gt,2000_2010_mean_err_gt,2010_2018_mean_flux_gt,2010_2018_mean_err_gt,level_0,index,area_km2_gained_total,perimeter_gained_total,perimeter_error_gained_total,area_km2_lost_total,perimeter_lost_total,perimeter_error_lost_total,area_km2_lost_early,perimeter_lost_early,perimeter_error_lost_early,area_km2_gained_early,perimeter_gained_early,perimeter_error_gained_early,area_km2_lost_late,perimeter_lost_late,perimeter_error_lost_late,area_km2_gained_late,perimeter_gained_late,perimeter_error_gained_late,Early_satellite_id,Middle_satellite_id,Late_satellite_id,Early_date,Mid_date,Late_date,Early_date_decimal_year,Middle_date_decimal_year,Late_date_decimal_year,total_years_elapsed,early_to_middle_years_elapsed,middle_to_late_years_elapsed,net_area_change_total,net_area_change_total_error,net_area_change_per_year_total,net_area_change_per_year_total_error,net_area_change_early,net_area_change_early_error,net_area_change_per_year_early,net_area_change_per_year_early_error,net_area_change_late,net_area_change_late_error,net_area_change_per_year_late,net_area_change_per_year_late_error,average_GlaThiDa_thickness_lost_early,average_GlaThiDa_thickness_gained_early,average_GlaThiDa_thickness_lost_late,average_GlaThiDa_thickness_gained_late,average_RUS_andrey_thickness_lost_early,average_RUS_andrey_thickness_gained_early,average_RUS_andrey_thickness_lost_late,average_RUS_andrey_thickness_gained_late,average_RUS_julian_thickness_lost_early,average_RUS_julian_thickness_gained_early,average_RUS_julian_thickness_lost_late,average_RUS_julian_thickness_gained_late,average_Farinotti_thickness_lost_early,average_Farinotti_thickness_gained_early,average_Farinotti_thickness_lost_late,average_Farinotti_thickness_gained_late,average_tomo_thickness_lost_early,average_tomo_thickness_gained_early,average_tomo_thickness_lost_late,average_tomo_thickness_gained_late,thickness_for_area_lost_early,thickness_for_area_gained_early,thickness_for_area_lost_late,thickness_for_area_gained_late,thickness_for_area_lost_early_err,thickness_for_area_gained_early_err,thickness_for_area_lost_late_err,thickness_for_area_gained_late_err,area_flux_gt_gained_early,area_flux_gt_gained_early_error,area_flux_gt_lost_early,area_flux_gt_lost_early_error,area_flux_gt_gained_late,area_flux_gt_gained_late_error,area_flux_gt_lost_late,area_flux_gt_lost_late_error,terminus_gt_change_total,terminus_gt_change_total_area_error,terminus_gt_change_per_year_total,terminus_gt_change_per_year_total_area_error,terminus_gt_change_per_year_early,terminus_gt_change_per_year_early_area_error,terminus_gt_change_per_year_late,terminus_gt_change_per_year_late_area_error,Frontal_ablation_2000_to_2010_gt_per_yr_mean,Frontal_ablation_2000_to_2010_gt_per_yr_mean_err,Frontal_ablation_2010_to_2020_gt_per_yr_mean,Frontal_ablation_2010_to_2020_gt_per_yr_mean_err,lon,lat,geometry,Region_name,Subregion_name,Subregion_name2
24,RGI60-01.14443,3.578656,4.825904,4.112021,5.698686,3.618520,4.984129,Hubbard Glacier,2834.493,0.0,5889.0,1911.0,12.0,151.0,131321.0,0.0,0.0,0.0,1.0,9.0,9.0,0.0,Alaska,520.175415,1.0,-2.164114e+06,2.531205e+06,316.332123,1029.734152,336.758739,35.000000,21.178654,23.823666,21.654292,23.951276,1193.616896,19.698272,1064.354064,13.009592,431,431,0,0,10.775,3.853491,0.571863,4.498209,0.727486,3.944315,0.626132,29,29,3.602733,20141.640226